In [3]:
import pandas as pd
import numpy as np
import collections
import math
import gc
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from transformers import AutoConfig , AutoModel , AutoTokenizer
import gc
import torch
import torch.nn as nn
# from apex import amp


2021-11-09 19:14:09.759577: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
new_data = pd.read_csv('input/ChaiiExtended.csv')

In [26]:
new_data['len_context'] = new_data['context'].apply(lambda x: len(x))

In [3]:
orignal_data = pd.read_csv('input/train_folds.csv')
# orignal_data['len_context'] = orignal_data['context'].apply(lambda x: len(x))

In [35]:
orignal_data = orignal_data.merge(new_data[["id","len_context"]], on ="id",how="left")

In [41]:
orignal_data['len_context_y'].fillna(-1, inplace=True)

In [42]:
orignal_data['len_context_y'] =orignal_data['len_context_y'].astype(int) 

In [44]:
orignal_data['flag'] = (orignal_data['len_context_y']==orignal_data['len_context_x'])

False    6615
True     1114
Name: flag, dtype: int64

In [58]:
orignal_data[orignal_data.flag==True].head(10)

,id,context,question,answer_text,answer_start,language,kfold,answers,len_context_x,len_context_y,flag,len
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,3,"{'answer_start': [53], 'text': ['206']}",4340,4340,True,4340
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0,"{'answer_start': [2358], 'text': ['காசுமீரில்']}",7647,7647,True,7608
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,4,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட...",4970,4970,True,4961
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,4,"{'answer_start': [68], 'text': ['தாலாட்டு']}",8493,8493,True,8489
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,4,"{'answer_start': [585], 'text': ['சூரியனும்']}",9954,9954,True,9945
5,d419db018,மின்னணுவியல் (Electronics) மின்னணுக்கள் அல்லது...,திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?,IBM,4171,tamil,2,"{'answer_start': [4171], 'text': ['IBM']}",8796,8796,True,8778
6,c29e29ab6,அலெக்ஸாண்டர் கிரகாம் பெல் ( Alexander Graham B...,தொலைபேசியைக் கண்டுபிடித்தவர் யார்?,அலெக்ஸாண்டர் கிரகாம் பெல்,0,tamil,2,"{'answer_start': [0], 'text': ['அலெக்ஸாண்டர் க...",12733,12733,True,12606
7,2a5ba78e2,முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...,முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?,1914ம்,1070,tamil,1,"{'answer_start': [1070], 'text': ['1914ம்']}",33997,33997,True,33911
8,10ff95f4c,இந்தியாவின் தேசிய மனித உரிமை ஆணையம் ஒரு தன்னா...,இந்தியாவில் மனித உரிமை ஆணையம் எப்போது நிறுவப்ப...,"அக்டோபர் 12, 1993",90,tamil,2,"{'answer_start': [90], 'text': ['அக்டோபர் 12, ...",4217,4217,True,4198
9,7a6e807d7,"நெல்சன் மண்டேலா (Nelson Rolihlahla Mandela, 18...",நெல்சன் மண்டேலா எத்தனை ஆண்டுகள் சிறையில் இருந்...,27,494,tamil,0,"{'answer_start': [494], 'text': ['27']}",13346,13346,True,13313


In [32]:
new_data['len_context'].values == orignal_data['len_context'].values

False

In [52]:
orignal_data.loc[569:575]

,id,context,question,answer_text,answer_start,language,kfold,answers,len_context_x,len_context_y,flag,len
569,272e353d2,"सतीश धवन अंतरिक्ष केंद्र, भारतीय अंतरिक्ष अनुस...",´सतीश धवन अंतरिक्ष केंद्र´ किस राज्य में स्थित...,आंध्र प्रदेश,91,hindi,4,"{'answer_start': [91], 'text': ['आंध्र प्रदेश']}",1426,1426,True,1424
570,22c8593ec,श्रीगुप्त (शासनकाल 240 – 280 ई)[1] गुप्त साम्र...,गुप्त राजवंश का संस्थापक कौन थे?,श्रीगुप्त,0,hindi,1,"{'answer_start': [0], 'text': ['श्रीगुप्त']}",593,593,True,592
571,fcf61ba87,Br\nब्रोमीन आवर्त सारणी (periodic table) के सप...,कौन-सी धातु द्रव अवस्था में पायी जाती है?,ब्रोमीन,3,hindi,3,"{'answer_start': [3], 'text': ['ब्रोमीन']}",3301,3301,True,3301
572,56e7ee1db,यह समिति संसद के माध्यम से सरकार द्वारा प्राप्...,प्राक्कलन समिति में कितने सदस्य होते हैं?,30,566,hindi,1,"{'answer_start': [566], 'text': ['30']}",810,810,True,810
573,2423eb462,अंटार्कटिका\nअंतरिक्ष से अंटार्कटिका का दृश्य।...,अंटार्कटिका का क्षेत्रफल कितना है?,"14,000,000 किमी2",160,hindi,1,"{'answer_start': [160], 'text': ['14,000,000 क...",12860,12860,True,12824
574,c3bdbe2cb,"नीलोफर, अक्टूबर २०१४ में दक्षिण हिंद महासागर म...",चक्रवात निलोफर कब हुआ था?,"२७ अक्टूबर, २०१४",254,hindi,3,"{'answer_start': [254], 'text': ['२७ अक्टूबर, ...",430,430,True,426
575,c718a0b49,अंकोरवाट (खमेर भाषा: អង្គរវត្ត) कंबोडिया में ए...,अंकोरवाट का निर्माण किसने किया था?,सम्राट सूर्यवर्मन द्वितीय,414,hindi,0,"{'answer_start': [414], 'text': ['सम्राट सूर्य...",7854,7854,True,7848


In [6]:
my_cleaned = pd.read_csv('input/cleaned_data.csv')

In [48]:
my_cleaned['len'] = my_cleaned['context'].apply(lambda x : len(x))

In [51]:
orignal_data = orignal_data.merge(my_cleaned[['id','len']],on="id",how="left")

In [2]:
import torch

In [3]:
from string import punctuation

In [4]:
import torch.nn.functional as F
import itertools

In [5]:
ids_to_remove = ['1a2160a69','bc9f0d533','632c16ba0']

In [6]:
train_folds = pd.read_csv('input/train_folds.csv')

In [7]:
## check data loader 

import config
import dataset
import preprocessing
from transformers import AutoTokenizer
from model import XLM_RoBertamodel
import engine




In [8]:

check = train_folds[train_folds.kfold==3]
# check = train_folds.head(1500)
check.answers = check.answers.apply(eval)
train_set, valid_set = check.head(200), check.tail(5)
    
train_features, valid_features = [],[]



In [9]:

tokenizer = AutoTokenizer.from_pretrained(config.TOKENIZER)

In [46]:
def get_features(example, tokenizer, inference=False):
    example["question"] = example["question"].lstrip()
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=config.MAX_SEQ_LEN,
        stride=config.DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []

    if not inference:

        sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
        offset_mapping = tokenized_example.pop("offset_mapping")
        
#         features['context'] = {example['context']}
        

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            feature = {}
            
            feature['context'] = example['context']
            feature['answer_text']= example['answer_text']
            feature["example_id"] = example['id']
            
            feature['sequence_ids'] = [
                0 if i is None else i for i in tokenized_example.sequence_ids(i)]
            feature['question'] = example['question']
            
            

            input_ids = tokenized_example["input_ids"][i]
            attention_mask = tokenized_example["attention_mask"][i]

            feature['input_ids'] = input_ids
            feature['attention_mask'] = attention_mask
            feature['offset_mapping'] = offsets

            cls_index = input_ids.index(tokenizer.cls_token_id)
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_example.sequence_ids(i)
            
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = example["answers"]
            
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    feature["start_position"] = cls_index
                    feature["end_position"] = cls_index
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    feature["start_position"] = token_start_index - 1
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    feature["end_position"] = token_end_index + 1
            features.append(feature)
    else:
        for i in range(len(tokenized_example["input_ids"])):
            feature = {}
            feature["example_id"] = example['id']
            feature['context'] = example['context']
            feature['question'] = example['question']
            feature['input_ids'] = tokenized_example['input_ids'][i]
            feature['attention_mask'] = tokenized_example['attention_mask'][i]
            feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
            feature['sequence_ids'] = [
                0 if i is None else i for i in tokenized_example.sequence_ids(i)]
            features.append(feature)

    return features


In [11]:
valid_set.reset_index(drop=True,inplace=True)

In [12]:
valid_set.tail()

,id,context,question,answer_text,answer_start,language,kfold,answers
0,a3068dd53,"भारत छोड़ो आन्दोलन, द्वितीय विश्वयुद्ध के समय ...",भारत छोड़ो आंदोलन कब शुरू हुआ था?,९ अगस्त १९४२,46,hindi,3,"{'answer_start': [46], 'text': ['९ अगस्त १९४२']}"
1,56d173d67,मानव व अन्य स्तनधारी प्राणियों मे कर्ण या कान ...,जीव विज्ञान में मानवीय कान के कितने भाग होते हैं?,तीन,938,hindi,3,"{'answer_start': [938], 'text': ['तीन']}"
2,6ac550541,नट (अंग्रेजी: Nat caste) उत्तर भारत में हिन्दू...,'नट'' उत्तर भारत में किस धर्म को मानने वाली एक...,हिन्दू,40,hindi,3,"{'answer_start': [40], 'text': ['हिन्दू']}"
3,2e7eca9e8,नंदवंश प्राचीन भारत का सर्वाधिक शक्तिशाली एवं ...,नंद वंश का संस्थापक कौन था?,सम्राट महापद्यनन्द,15605,hindi,3,"{'answer_start': [15605], 'text': ['सम्राट महा..."
4,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi,3,"{'answer_start': [155], 'text': ['17']}"


In [13]:
train_features = []
valid_features = []
for i, row in train_set.iterrows():
    train_features += preprocessing.get_features(row, tokenizer)

for i, row in valid_set.iterrows():
    valid_features += preprocessing.get_features(row, tokenizer)
 
    

In [14]:
train_features[0].keys()

dict_keys(['context', 'answer_text', 'example_id', 'sequence_ids', 'question', 'input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])

In [15]:
valid_features[0].keys()

dict_keys(['context', 'answer_text', 'example_id', 'sequence_ids', 'question', 'input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])

In [16]:
train_dataloader = dataset.Datasetloader(features=train_features, test=False,inference=False).fetch(
        batch_size=32, num_workers=config.NUM_WORKERS, shuffle=False, drop_last=False)


valid_dataloader = dataset.Datasetloader(features=valid_features, test=True,inference=False).fetch(
        batch_size=32, num_workers=config.NUM_WORKERS, shuffle=False, drop_last=False)

In [17]:
len(valid_dataloader) , len(valid_features),  valid_set.shape 

(2, 38, (5, 8))

## Model Architecture

#### Check max  pooling and min pooling in the model

In [38]:
model_config = AutoConfig.from_pretrained(config.CONFIG_NAME)
model = AutoModel.from_pretrained( config.MODEL_PATH, config=model_config)


# device = torch.device("cpu")  #
# model_config = AutoConfig.from_pretrained(config.CONFIG_NAME)
# model = XLM_RoBertamodel(config.MODEL_PATH, config=model_config)
# model = nn.DataParallel(model)

# model.cuda();
# # model.to(device)
# model.load_state_dict(torch.load('output/xlm_roberta_v3/checkpoint-fold-0/pytorch_model.bin',map_location=torch.device('cuda')));


# model.eval()

Some weights of the model checkpoint at xlm-roberta/deepset/xlm-roberta-large-squad2/ were not used when initializing XLMRobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [140]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
batch = next(iter(valid_dataloader))

In [158]:
start_logits_cls,end_logits_cls = [],[]
start_logits_pool,end_logits_pool = [],[]
ans_pred = []
with torch.no_grad():
        
    for data in tqdm(valid_dataloader, total=len(valid_dataloader)):
        for k, v in data.items():
            if isinstance(v,list):
                data[k] = v
            else:
                data[k] = v.to('cpu')

        outputs = model(data['input_ids'],data['attention_mask'])
        pooler_output = outputs[1]

        last_hidden_state = outputs[0]
        cls_embeddings = last_hidden_state[:, 0]
        
        logits = nn.Linear(model_config.hidden_size, 2)(pooler_output)
        logits_cls = nn.Linear(model_config.hidden_size, 2)(last_hidden_state)
        
        
        
        start_logits_cl, end_logits_cl = logits_cls.split(1, dim=-1)
        start_logits_pol, end_logits_pol = logits.split(1, dim=-1)
        
        start_logits_cls.append(start_logits_cl.cpu().numpy().tolist())
        end_logits_cls.append(end_logits_cl.cpu().numpy().tolist())
        
        start_logits_pool.append(start_logits_pol.cpu().numpy().tolist())
        end_logits_pool.append(end_logits_pol.cpu().numpy().tolist())
        
        


100%|██████████| 12/12 [00:57<00:00,  4.78s/it]


In [39]:
outputs = model(batch['input_ids'],batch['attention_mask'])

In [43]:
outputs['last_hidden_state'].shape

torch.Size([32, 384, 1024])

In [109]:
pooler_output = outputs[1]

last_hidden_state = outputs[0]
cls_embeddings = last_hidden_state[:, 0]


In [110]:
last_hidden_state[1].shape

torch.Size([384, 1024])

In [112]:
 # regression head


In [113]:
s

In [59]:
logits

tensor([[ 0.2065, -0.2490],
        [ 0.2769, -0.3237],
        [ 0.1607, -0.1827],
        [ 0.1137, -0.4239],
        [ 0.1774, -0.3180],
        [ 0.2676, -0.2725],
        [ 0.1007, -0.4610],
        [-0.0693, -0.1749],
        [ 0.1464, -0.4047],
        [ 0.2477, -0.2748],
        [ 0.1010, -0.4305],
        [ 0.1768, -0.2771],
        [ 0.2080, -0.2621],
        [ 0.1540, -0.3558],
        [ 0.1898, -0.3208],
        [ 0.2592, -0.3322],
        [ 0.2748, -0.2054],
        [ 0.2040, -0.2322],
        [ 0.1727, -0.3757],
        [ 0.1551, -0.2486],
        [ 0.1463, -0.3688],
        [ 0.2117, -0.2839],
        [ 0.0626, -0.2578],
        [ 0.2045, -0.4312],
        [ 0.2170, -0.2493],
        [ 0.0322, -0.2141],
        [ 0.1715, -0.4871],
        [ 0.2164, -0.3469],
        [ 0.1787, -0.3039],
        [ 0.2195, -0.4510],
        [ 0.1902, -0.3005],
        [ 0.0960, -0.4765]], grad_fn=<AddmmBackward>)

In [162]:


jacc   = preprocessing.postprocess_qa_predictions(examples=valid_set, features=valid_features,raw_predictions=(np.vstack(start_logits_pool),np.vstack(end_logits_pool)) ,tokenizer = tokenizer, n_best_size = 20, max_answer_length = 30)





Post-processing 32 example predictions split into 373 features.


In [160]:


jacc   = preprocessing.postprocess_qa_predictions(examples=valid_set, features=valid_features,raw_predictions=(np.vstack(start_logits_cls),np.vstack(end_logits_cls)) ,tokenizer = tokenizer, n_best_size = 20, max_answer_length = 30)





Post-processing 32 example predictions split into 373 features.


TypeError: '>=' not supported between instances of 'list' and 'int'

In [161]:
np.vstack(start_logits_cls)

array([[[-0.37604973],
        [ 0.81268066],
        [ 0.90655112],
        ...,
        [ 0.60618645],
        [ 1.20463145],
        [ 1.16679204]],

       [[-0.14492014],
        [ 0.87696058],
        [ 0.98381966],
        ...,
        [ 1.21595979],
        [ 1.18609083],
        [ 1.13837183]],

       [[-0.05872491],
        [ 0.86531037],
        [ 1.03937352],
        ...,
        [ 0.98119479],
        [ 1.44031656],
        [ 1.16028333]],

       ...,

       [[-0.26933268],
        [-0.02453575],
        [ 0.39442694],
        ...,
        [ 0.14442688],
        [ 0.15570727],
        [ 0.14565051]],

       [[-0.27930233],
        [ 0.03571079],
        [ 0.35002375],
        ...,
        [ 0.10036696],
        [ 0.24820197],
        [ 0.23289579]],

       [[-0.23200926],
        [ 0.05761795],
        [ 0.32836941],
        ...,
        [ 0.2412253 ],
        [ 0.2412253 ],
        [ 0.2412253 ]]])

In [40]:
outputs['pooler_output'].shape

(torch.Size([32, 1024]), torch.Size([32, 1024]))

In [29]:
c = 0 
for data in tqdm(valid_dataloader, total=len(valid_dataloader)):  
    for k, v in data.items():
        print('reached here')
        if isinstance(v,list):
#         if type(v) == list:
            data[k] = v
#             print(k,len(v))
#             if k == 'offset_mapping':
#                 print(v)
        else:
#             print(k,v.shape)
            data[k] =v
#             data[k] = v.to(config.DEVICE)

  0%|          | 0/9 [00:00<?, ?it/s]


TypeError: an integer is required (got type NoneType)

In [22]:
# to do : load the model from the dir and kind of create an inference for the model 

from model import XLM_RoBertamodel




In [39]:
class XLM_RoBertamodel(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(XLM_RoBertamodel, self).__init__()
        self.config = config
        config.update({
            "layer_norm_eps": 1e-7,
            "output_hidden_states": True
            }) 
        self.xlm_roberta = AutoModel.from_pretrained(
            modelname_or_path, config=config)
        
        
        
        self.high_dropout = nn.Dropout(config.hidden_dropout_prob)
        self.qa_outputs = nn.Linear(config.hidden_size * 2, 2)

        self._init_weights(self.qa_outputs)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self,
                input_ids,
                attention_mask=None,
                start_position=None,
                end_position=None,
                offset_mapping=None,
                
                # token_type_ids=None
                ):
        
        
        out = self.xlm_roberta(input_ids,attention_mask=attention_mask,)
        LAST_HIDDEN_LAYERS = 12

        out = out.hidden_states 
        out = torch.stack(tuple(out[-i - 1] for i in range(LAST_HIDDEN_LAYERS)), dim=0)
       
        out_mean = torch.mean(out, dim=0) 
        out_max, _ = torch.max(out, dim=0)
        out = torch.cat((out_mean, out_max), dim=-1) 


        # Multisample Dropout: https://arxiv.org/abs/1905.09788
        logits = torch.mean(torch.stack([self.qa_outputs(self.high_dropout(out))for _ in range(5)], dim=0), dim=0)

        start_logits, end_logits = logits.split(1, dim=-1)

       
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        loss = None
        if (start_position is not None) & (end_position is not None):
            loss = loss_fn((start_logits, end_logits),
                           (start_position, end_position))
            loss = loss / config.GRADIENT_ACC_STEPS

        return start_logits, end_logits, loss
    
    
def loss_fn(preds, labels):
    
    start_preds, end_preds = preds
    start_labels, end_labels = labels

    start_loss = nn.CrossEntropyLoss(
        ignore_index=-1)(start_preds, start_labels)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_preds, end_labels)
    total_loss = (start_loss + end_loss)/2

    return total_loss




In [18]:
class XLM_RoBertamodel(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(XLM_RoBertamodel, self).__init__()
        self.config = config
        self.xlm_roberta = AutoModel.from_pretrained(
            modelname_or_path, config=config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        
        
        
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self._init_weights(self.qa_outputs)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self,
                input_ids,
                attention_mask=None,
                start_position=None,
                end_position=None,
                offset_mapping=None,
                
                # token_type_ids=None
                ):

        outputs = self.xlm_roberta(input_ids, attention_mask=attention_mask,)
        sequence_output = outputs[0]
        pooled_output = outputs[1]

        # sequence_output = self.dropout(sequence_output)
        qa_logits = self.qa_outputs(sequence_output)

        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        loss = None
        if (start_position is not None) & (end_position is not None):
            loss = loss_fn((start_logits, end_logits),
                           (start_position, end_position))
            loss = loss / config.GRADIENT_ACC_STEPS

        return start_logits, end_logits, loss


In [38]:
class XLM_RoBertamodel(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(XLM_RoBertamodel, self).__init__()
        self.config = config
        config.update({
            "layer_norm_eps": 1e-7,
            "output_hidden_states": True
            }) 
        self.xlm_roberta = AutoModel.from_pretrained(
            modelname_or_path, config=config)
        
        
        
        self.high_dropout = nn.Dropout(config.hidden_dropout_prob)
        self.qa_outputs = nn.Linear(config.hidden_size * 2, 2)

        self._init_weights(self.qa_outputs)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self,
                input_ids,
                attention_mask=None,
                start_position=None,
                end_position=None,
                offset_mapping=None,
                
                # token_type_ids=None
                ):

        
        
        
        out = self.xlm_roberta(input_ids,attention_mask=attention_mask,)
        LAST_HIDDEN_LAYERS = 12

        out = out.hidden_states 
        out = torch.stack(tuple(out[-i - 1] for i in range(LAST_HIDDEN_LAYERS)), dim=0)
        
        
       
        out_mean = torch.mean(out, dim=0) 
        out_max, _ = torch.max(out, dim=0)
        out = torch.cat((out_mean, out_max), dim=-1) 


        # Multisample Dropout: https://arxiv.org/abs/1905.09788
        logits = torch.mean(torch.stack([
            self.qa_outputs(self.high_dropout(out))
            for _ in range(5)
        ], dim=0), dim=0)

        start_logits, end_logits = logits.split(1, dim=-1)

       
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        loss = None
        if (start_position is not None) & (end_position is not None):
            loss = loss_fn((start_logits, end_logits),
                           (start_position, end_position))
            loss = loss / config.GRADIENT_ACC_STEPS

        return start_logits, end_logits, loss
    
    


In [35]:
# def loss_fn(preds, labels):
    
#     start_preds, end_preds = preds
#     start_labels, end_labels = labels

#     start_loss = nn.CrossEntropyLoss(
#         ignore_index=-1)(start_preds, start_labels)
#     end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_preds, end_labels)
#     total_loss = (start_loss + end_loss)/2

#     return total_loss



# def loss_fn(preds, labels):
    
#     start_preds, end_preds = preds
#     start_labels, end_labels = labels
# #     m = torch.nn.LogSoftmax(dim=1)
#     loss_fct = torch.nn.KLDivLoss()
#     print(f'the shape of start_preds_is {start_preds.shape} and the values are as follows {start_preds}')
    
    
#     print(f'the logits value after softmax is the following ,{F.log_softmax(start_preds,dim=1)}  ')
#     print(f' the shape of lable is. {start_labels.shape} and the value of shape is {start_labels}')
    
#     start_loss = loss_fct(F.log_softmax(start_preds,dim=1), start_labels)
#     end_loss = loss_fct(F.log_softmax(end_preds,dim=1), end_labels)
#     total_loss = (start_loss + end_loss)
#     return total_loss


def pos_weight(pred_tensor, pos_tensor, neg_weight=1, pos_weight=1):
    # neg_weight for when pred position < target position
    # pos_weight for when pred position > target position
    gap = torch.argmax(pred_tensor, dim=1) - pos_tensor
    gap = gap.type(torch.long)
    return torch.where(gap < 0, -neg_weight * gap, pos_weight * gap)


    
def loss_fn(preds, labels):
    
    start_logits, end_logits = preds
    start_position, end_position = labels
    
    
    loss_fct = nn.CrossEntropyLoss(reduce='none')
    
    start_loss = loss_fct(start_logits, start_position) * pos_weight(start_logits, start_position, 1, 1)
    end_loss = loss_fct(end_logits, end_position) * pos_weight(end_logits, end_position, 1, 1)
    
    start_loss = torch.mean(start_loss)
    end_loss = torch.mean(end_loss)
    
    total_loss = (start_loss + end_loss)/2
    return total_loss


In [36]:

# Intialize Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_config = AutoConfig.from_pretrained(config.CONFIG_NAME)
model = XLM_RoBertamodel(config.MODEL_PATH, config=model_config)



# optimizer
optimizer = engine.get_optimizer(model, type="a")



RUN_PARALLEL = True
if RUN_PARALLEL :
    print('Model pushed to {} GPU(s), type {}.'.format(
        torch.cuda.device_count(), 
        torch.cuda.get_device_name(0))
    )
    model = nn.DataParallel(model)
#         model = model.cuda() 

model.to(device)

# scheduler
num_training_steps = math.ceil(
    len(train_dataloader) / config.GRADIENT_ACC_STEPS) * config.EPOCHS
if config.WARMUP_RATIO > 0:
    num_warmup_steps = int(config.WARMUP_RATIO * num_training_steps)
else:
    num_warmup_steps = 0
print(
    f"Total Training Steps: {num_training_steps}, Total Warmup Steps: {num_warmup_steps}")

scheduler = engine.get_scheduler(
    optimizer, num_warmup_steps, num_training_steps)




Some weights of the model checkpoint at xlm-roberta/deepset/xlm-roberta-large-squad2/ were not used when initializing XLMRobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model pushed to 8 GPU(s), type Tesla V100-SXM3-32GB.
Total Training Steps: 840, Total Warmup Steps: 84


In [37]:
def train_fn(dataloader, model, optimizer, device, scheduler):

    count = 0
#     losses = AverageMeter()
    model.zero_grad()
    model.train()
   

    for batch_idx, data in enumerate(tqdm(dataloader, total=len(dataloader))):
        for k, v in data.items():
            data[k] = v.to(device)

        _, _, loss = model(**data)

        
        if config.FP16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
        else:
            loss.sum().backward()

        count += data['input_ids'].size(0)
        
#         losses.update(loss.mean().item(), data['input_ids'].size(0))

        if batch_idx % config.GRADIENT_ACC_STEPS == 0 or batch_idx == len(dataloader) - 1:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()


    return loss.mean().item()


def evaluate_modf(dataloader, model, device,inference=False):
    model.eval()
    start_logits ,end_logits = [],[]
    with torch.no_grad():
        
        if inference:
            for data in tqdm(dataloader, total=len(dataloader)):
                for k, v in data.items():
                    data[k] = v.to(device)

                start_logits, end_logits, _ = model(**data)
                predictions.append([output.detach().cpu().numpy()
                                   for output in (start_logits, end_logits)])

        else:
            for data in tqdm(dataloader, total=len(dataloader)):
                for k, v in data.items():
                    if isinstance(v,list):
                        data[k] = v
                    else:
                        data[k] = v.to(device)
                    
                outputs_start, outputs_end, loss = model(data['input_ids'],data['attention_mask'],data['start_position'],data['end_position'])
                
                
                
                start_logits.append(outputs_start.cpu().numpy().tolist())
                end_logits.append(outputs_end.cpu().numpy().tolist())
                
                del outputs_start,outputs_end
                gc.collect()
                
                            
    return loss.mean().item(), np.vstack(start_logits), np.vstack(end_logits)

In [38]:
import copy

In [39]:

for epoch in range(4):
        train_loss = train_fn(
            train_dataloader, model, optimizer, device, scheduler)
        
        test_loss, start_logits , end_logits = evaluate_modf(
            valid_dataloader, model, device)
        
        
#         valid_features_ =  copy.deepcopy(valid_features)
#         jaccard   = preprocessing.postprocess_qa_predictions(examples=valid_set, features=valid_features_,raw_predictions=(start_logits,end_logits) ,tokenizer = tokenizer, n_best_size = 20, max_answer_length = 30)
#         del start_logits ,end_logits ,valid_features_
#         gc.collect()
        
        
        print(f"EPOCH : {epoch + 1}/{4}")
        
        print(f"| Train Loss = {train_loss} | Valid Loss = {test_loss}| ")
#         print(f"| Train Loss = {train_loss} | Valid Loss = {test_loss}| Jaccard Score = {jaccard}|")


100%|██████████| 2/2 [00:00<00:00,  2.37it/s]


EPOCH : 1/4
| Train Loss = 0.0 | Valid Loss = 35.55387878417969| 


100%|██████████| 2/2 [00:00<00:00,  2.31it/s]


EPOCH : 2/4
| Train Loss = 0.0 | Valid Loss = 14.527071952819824| 


100%|██████████| 2/2 [00:00<00:00,  2.27it/s]


EPOCH : 3/4
| Train Loss = 0.0 | Valid Loss = 6.631829261779785| 


100%|██████████| 2/2 [00:00<00:00,  2.36it/s]

EPOCH : 4/4
| Train Loss = 0.0 | Valid Loss = 40.824737548828125| 


In [67]:
valid_features[0]['offset_mapping']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 (0, 2),
 (2, 3),
 (3, 5),
 (5, 7),
 (7, 9),
 (9, 10),
 (10, 14),
 (14, 18),
 (18, 20),
 (20, 22),
 (22, 24),
 (24, 25),
 (25, 32),
 (32, 33),
 (33, 34),
 (34, 35),
 (35, 39),
 (39, 42),
 (42, 45),
 (45, 46),
 (46, 49),
 (49, 52),
 (52, 55),
 (55, 58),
 (58, 59),
 (59, 61),
 (61, 63),
 (63, 66),
 (66, 69),
 (69, 73),
 (73, 77),
 (77, 80),
 (80, 81),
 (81, 85),
 (85, 86),
 (86, 90),
 (90, 93),
 (93, 97),
 (97, 101),
 (101, 104),
 (104, 108),
 (108, 112),
 (112, 116),
 (116, 117),
 (117, 123),
 (123, 128),
 (128, 131),
 (131, 132),
 (132, 136),
 (136, 138),
 (138, 139),
 (139, 144),
 (144, 147),
 (147, 151),
 (151, 152),
 (152, 160),
 (160, 163),
 (163, 170),
 (170, 173),
 (173, 174),
 (174, 177),
 (177, 181),
 (181, 185),
 (185, 188),
 (188, 196),
 (196, 199),
 (199, 200),
 (200, 202),
 (202, 204),
 (204, 206),
 (206, 207),
 (207, 210),
 (210, 213),
 (213, 224),
 (224, 228),
 (228, 229),
 (229, 232),
 (2

In [22]:
train_loss

72.5696792602539

In [51]:
c = [1,3,4,5]
d = list(c)

In [52]:
d[1] = 0

In [53]:
d

[1, 0, 4, 5]

In [54]:
c

[1, 3, 4, 5]

In [17]:
start_logits,end_logits = [],[]
jaccard_score = 0.0
ans_pred = []
with torch.no_grad():
        
    for data in tqdm(valid_dataloader, total=len(valid_dataloader)):
        for k, v in data.items():
            if isinstance(v,list):
                data[k] = v
            else:
                data[k] = v.to(device)

        outputs_start, outputs_end, loss = model(data['input_ids'],data['attention_mask'],data['start_position'],data['end_position'])
        start_logits.append(outputs_start.cpu().numpy().tolist())
        end_logits.append(outputs_end.cpu().numpy().tolist())
        


100%|██████████| 507/507 [08:18<00:00,  1.02it/s] 


In [18]:
jacc   = preprocessing.postprocess_qa_predictions(examples=valid_set, features=valid_features,raw_predictions=(np.vstack(start_logits),np.vstack(end_logits)) ,tokenizer = tokenizer, n_best_size = 20, max_answer_length = 30)

Post-processing 1000 example predictions split into 8097 features.


In [19]:
jacc

0.692498903203219

In [20]:
train_loss

11.993694305419922

In [45]:
train_loss

5.7059454917907715

In [16]:
def get_predictions(checkpoint_path,tokenizer):    
    
    model_config = AutoConfig.from_pretrained(config.CONFIG_NAME)
    model = XLM_RoBertamodel(config.MODEL_PATH, config=model_config)
    model = nn.DataParallel(model)
    
    model.cuda();
    model.load_state_dict(torch.load((base_model_path + checkpoint_path) ,map_location=torch.device('cuda')));
    
    start_logits = []
    end_logits = []
    jaccard_score = 0.0
    
    true_vals,context_val,prediction_vals=[],[],[]
    for batch in valid_dataloader:
        with torch.no_grad():
            outputs_start, outputs_end,_ = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            
                    
            jaccard_score += new_get_jaccard_score(batch, outputs_start,outputs_end,tokenizer,n_best_size = 20,max_answer_length = 30)
                
            
                        
            true_vals.append(batch['answer_text'])
            context_val.append(batch['context'])
          
            #del  outputs_end,outputs_start
    del model, model_config
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits),true_vals,context_val,outputs_start,outputs_end, jaccard_score/len(valid_dataloader)

In [182]:
device = torch.device("cpu")  #
model_config = AutoConfig.from_pretrained(config.CONFIG_NAME)
model = XLM_RoBertamodel(config.MODEL_PATH, config=model_config)
model = nn.DataParallel(model)

model.cuda();
# model.to(device)
model.load_state_dict(torch.load('output/xlm_roberta_v7/checkpoint-fold-3/pytorch_model.bin',map_location=torch.device('cuda')));


model.eval()

Some weights of the model checkpoint at xlm-roberta/deepset/xlm-roberta-large-squad2/ were not used when initializing XLMRobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DataParallel(
  (module): XLM_RoBertamodel(
    (xlm_roberta): XLMRobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(250002, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear

In [269]:
start_logits,end_logits = [],[]
outputs_start_ , outputs_end_ = [], []
jaccard_score = 0.0
ans_pred = []
with torch.no_grad():
        
    for data in tqdm(valid_dataloader, total=len(valid_dataloader)):
        for k, v in data.items():
            if isinstance(v,list):
                data[k] = v
            else:
                data[k] = v.to(device)

        outputs_start, outputs_end, loss = model(data['input_ids'],data['attention_mask'],data['start_position'],data['end_position'])
        start_logits.append(outputs_start.cpu().numpy().tolist())
        end_logits.append(outputs_end.cpu().numpy().tolist())
        
        outputs_start_.append(torch.softmax(outputs_start, dim=-1).cpu().detach().numpy())
        outputs_end_.append(torch.softmax(outputs_end, dim=-1).cpu().detach().numpy())


#         js, answer_prediction = _get_jaccard_score(data, outputs_start,outputs_end,tokenizer,n_best_size = 20,max_answer_length = 30)
#         jaccard_score += js
#         ans_pred.append(answer_prediction)
        
#         jaccard_score += _get_jaccard_score(data, outputs_start,outputs_end,tokenizer,n_best_size = 20,max_answer_length = 30)


100%|██████████| 12/12 [00:04<00:00,  2.80it/s]


In [257]:
cc = torch.softmax(outputs_start, dim=-1).cpu().detach().numpy()

In [259]:
cc1= torch.softmax(outputs_start, dim=0).cpu().detach().numpy()

In [263]:
len(cc1) ,len(cc)

21

In [194]:
char_pred_test_start = []
char_pred_test_end = []      

In [203]:
def token_level_to_char_level(text, offsets, preds):
    probas_char = np.zeros(len(text))
    for i, offset in enumerate(offsets):
        if offset[0] or offset[1]:
            probas_char[offset[0]:offset[1]] = preds[i]

    return probas_char


In [202]:
valid_set.shape

(32, 8)

In [235]:
valid_features[0]['offset_mapping'][1]

(0, 2)

In [236]:
cc[0:2] = outputs_start_[0][0][1]

In [237]:
cc[0:2]

array([2.61269161e-05, 2.61269161e-05])

In [250]:
np.argmax(probs_e)

490

In [251]:
valid_features[0]['context'][0:490]

"डॉ॰ रेड्डीज लेबोरेटरीज लिमिटेड (Dr. Reddy's Laboratories Ltd.), जिसे डॉ रेड्डीज के नाम से ट्रेड किया जाता है, आज भारत की दूसरी सबसे बड़ी औषधि कंपनी है। इसकी स्थापना 1984 में डॉक्टर के. अंजी रेड्डी ने की थी। इसके पूर्व डॉ॰ अंजी रेड्डी भारत सरकार के उपक्रम इंडियन ड्रग्स एंड फार्मास्यूटिकल्स लिमिटेड में कार्यरत थे। डॉ॰ रेड्डीज अनेकों प्रकार की दवायें बना कर देश-विदेश में विपणन करती है। कंपनी के पास आज 190 से अधिक दवाएं, दवाओं के निर्माण की करीब साठ सक्रिय सामग्रियां, नैदानिक उपकरण तथा सघन"

In [201]:
len(valid_features)

373

In [205]:
valid_features[0].keys()

dict_keys(['context', 'answer_text', 'example_id', 'sequence_ids', 'question', 'input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])

In [222]:
len(valid_features[0])

10

In [213]:
len(outputs_start_[0][0]) , len(valid_features[0]['offset_mapping'])

(384, 384)

In [215]:
probs=  token_level_to_char_level(valid_features[0]['context'], valid_features[0]['offset_mapping'], outputs_start_[0][0]) 

In [249]:
probs_e=  token_level_to_char_level(valid_features[0]['context'], valid_features[0]['offset_mapping'], outputs_end_[0][0]) 

In [221]:
probs

AttributeError: 'numpy.float64' object has no attribute 'idx'

In [ ]:
for px, tweet in enumerate(orig_tweet):
    char_pred_test_start.append(
        utils.token_level_to_char_level(tweet, offsets[px], outputs_start[px]))
    char_pred_test_end.append(
        utils.token_level_to_char_level(tweet, offsets[px], outputs_end[px]))

In [18]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()
    
    valid_ans_all = []

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        truth_ans = example["answer_text"]
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            
            
            start_prob = F.softmax(torch.tensor(start_logits),dim=0).cpu().numpy()
            end_prob = F.softmax(torch.tensor(end_logits),dim=0).cpu().numpy()

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char],
#                             "start_index" : start_index,
#                             "end_index" : end_index,
#                             "prob" : 0.5* (start_prob[start_index]+ end_prob[end_index])
                        }
                    )
            valid_ans_all.append(
               {
                example["id"] : valid_answers  
               } )        
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = jaccard(best_answer["text"],truth_ans)
        
        
    return predictions,valid_ans_all

def safe_div(x,y):
    if y == 0:
        return 1
    return x / y

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return safe_div(float(len(c)) , (len(a) + len(b) - len(c)))




In [21]:
pred  , valid_ans = postprocess_qa_predictions(examples=valid_set, features=valid_features,raw_predictions=(np.vstack(start_logits),np.vstack(end_logits)) , n_best_size = 20, max_answer_length = 30)

Post-processing 500 example predictions split into 7149 features.


In [19]:
jacc   = preprocessing.postprocess_qa_predictions(examples=valid_set, features=valid_features,raw_predictions=(np.vstack(start_logits),np.vstack(end_logits)) ,tokenizer = tokenizer, n_best_size = 20, max_answer_length = 30)

Post-processing 500 example predictions split into 7149 features.


In [20]:
jacc

0.6459937895437895

In [46]:
pd.DataFrame([pred]).apply(lambda x: np.mean(x),axis=1)

0    0.716209
dtype: float64

In [51]:
np.mean(pd.DataFrame([pred]),axis=1)[0]

0.7162094901588673

In [239]:
valid_set['prediction'] =  valid_set['id'].map(pred)
valid_set['predicted_jacc']=valid_set.apply(lambda row : jaccard(row['answer_text'],row['prediction']),axis=1)

In [251]:
valid_set['prediction_jem'] =  valid_set['id'].map(_get_jem_score(valid_ans,n=4))

In [259]:

valid_set['p_jem_jacc'] = valid_set.apply(lambda row : jaccard(row['answer_text'],row['prediction_jem']),axis=1)

In [247]:
valid_set['predicted_jacc'].mean() ,valid_set['p_jem_jacc'].mean()

(0.7162094901588673, 0.6959669878681265)

In [262]:
valid_set[valid_set.predicted_jacc !=valid_set.p_jem_jacc]

,id,context,question,answer_text,answer_start,language,kfold,answers,prediction,predicted_jacc,p_jem_jacc,prediction_jem
7,2a5ba78e2,முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...,முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?,1914ம்,1070,tamil,1,"{'answer_start': [1070], 'text': ['1914ம்']}",1914,0.000000,0.250000,1914ம் ஆண்டு முதல் 1918ம் ஆண்டு
16,b0dae4743,அனைத்துலக முறை அலகுகள்(SI) குழுமத்தினரால் நிர்...,1 நிமிடத்தில் எத்தனை விநாடிகள் உள்ளன?,60 நொடிகள்,909,tamil,1,"{'answer_start': [909], 'text': ['60 நொடிகள்']}",3600,0.000000,0.500000,60
23,89ea74032,புல்லாங்குழல் (புல்லாங்குழல் இசைக்கோப்பு) மிகவ...,புல்லாங்குழல் எந்த மரத்தில் செய்யப்படுகிறது?,மூங்கில்,6339,tamil,4,"{'answer_start': [6339], 'text': ['மூங்கில்']}",மூங்கிலால்,0.000000,1.000000,மூங்கில்
36,7d2ed1f64,"அ. இர. ரகுமான் (அல்லா இரக்கா இரகுமான், பிறப்பு...",2009ஆம் ஆண்டில் 2 ஆஸ்கார் விருதை வென்ற இந்திய ...,அ. இர. ரகுமான்,0,tamil,0,"{'answer_start': [0], 'text': ['அ. இர. ரகுமான்']}",அ. இர. ரகுமான்,1.000000,0.666667,இர. ரகுமான்
39,fd235557c,செம்பருத்தி அல்லது செவ்வரத்தை (Hibiscus rosa-s...,செம்பருத்தியின் உயிரியலில் பெயர் என்ன?,Hibiscus rosa-sinensis,31,tamil,1,"{'answer_start': [31], 'text': ['Hibiscus rosa...",செவ்வரத்தை,0.000000,0.250000,செவ்வரத்தை (Hibiscus rosa-sinensis
...,...,...,...,...,...,...,...,...,...,...,...,...
3480,2367,चीनी अक्षरों को जापानी शब्दों को लिखने के लिए ...,उस लेखन शैली का नाम क्या है जो हीरागाना और कात...,कानजी,668,hindi,-1,"{'answer_start': [668], 'text': ['कानजी']}",कानजी,1.000000,0.250000,"काना कहते हैं, कानजी"
3481,2368,"क्योइकु कानजी (教育漢字, अर्थात ""शिक्षा कानजी"") १,...",क्योइकु कान्जी में कितने अक्षर हैं?,"१,००६",44,hindi,-1,"{'answer_start': [44], 'text': ['१,००६']}","१,००६",1.000000,0.500000,"१,००६ अक्षरों"
3484,2371,जापनी विद्यार्थियों को बुनियादी १००६ कानजी अक्...,जापानी स्कूली बच्चों से क्या उम्मीद की जाती है?\n,बुनियादी १००६ कानजी अक्षर,23,hindi,-1,"{'answer_start': [23], 'text': ['बुनियादी १००६...",बुनियादी १००६ कानजी अक्षर,1.000000,0.666667,बुनियादी १००६ कानजी अक्षर (क्योइकु कानजी)
3490,2377,19वीं शताब्दी में मुद्रण को व्यापक रूप से अपना...,मुद्रण को व्यापक रूप से अपनाया जाने से पहले कु...,पांडुलिपियों,116,hindi,-1,"{'answer_start': [116], 'text': ['पांडुलिपियों']}",कॉलिग्राफर्स और प्रतिवादियों द्वारा बनाई गई प...,0.142857,0.125000,कॉलिग्राफर्स और प्रतिवादियों द्वारा बनाई गई प...


In [207]:
def _get_jem(eval_df,n=4):

    prob_score = []
    for prob , jacc_comb in zip (eval_df['prob'].tolist() *n ,list(itertools.product(eval_df['text'].values,eval_df['text'].values)) ):
        str1 , str2 = jacc_comb
        score = jaccard(str1 , str2)
        prob_score.append(prob*score)

    return [sum(prob_score[i:i+n]) for i in range(0,len(prob_score),n)]

In [118]:
# ids = {k for d in valid_ans for k in d.keys()}

In [241]:


def _get_jem_score(valid_ans,n=4):
    jem_predictions= collections.OrderedDict()
    for i in range(len(valid_ans)):

        k =list(valid_ans[i].keys())[0]

        eval_ans_id_sel = sorted(valid_ans[i].get(k), key=lambda x: x["score"], reverse=True)[:4]

        eval_df = pd.DataFrame(eval_ans_id_sel)
        eval_df['jem_score'] = _get_jem(eval_df,n=n)
        eval_df.sort_values(by='jem_score',ascending=False,inplace=True,ignore_index=True)

        jem_predictions[k] =  eval_df['text'][0]
        
    return jem_predictions



In [145]:
eval_df

,score,text,start_index,end_index,prob,jem_score
0,14.089586,12.6%,49,50,0.883674,1.752562
1,12.631596,12.6%,289,290,0.868889,1.752562
2,15.129937,"1,250",238,239,0.967658,0.967658
3,12.655212,50,248,248,0.834743,0.834743


In [168]:
for i in range(len(valid_ans)):
    if valid_ans[i].get('1473492ab'):
        eval_ans_id = valid_ans[i].get('1473492ab')

In [169]:
eval_ans_id

[{'score': 13.138822555541992,
  'text': ' 1960',
  'start_index': 122,
  'end_index': 122,
  'prob': 0.8369724266622645},
 {'score': 11.97038221359253,
  'text': ' 1960 के दशक',
  'start_index': 122,
  'end_index': 124,
  'prob': 0.6034223452136326},
 {'score': 10.8192138671875,
  'text': ' 1960 के दशक के मध्य',
  'start_index': 122,
  'end_index': 126,
  'prob': 0.531393499340414},
 {'score': 9.912975311279297,
  'text': ' 1960 के दशक के मध्य तक',
  'start_index': 122,
  'end_index': 127,
  'prob': 0.5115375728067632},
 {'score': 8.943208575248718,
  'text': ' 1960 के',
  'start_index': 122,
  'end_index': 123,
  'prob': 0.5031801737236297},
 {'score': 8.126932859420776,
  'text': ' 1960 के दशक के',
  'start_index': 122,
  'end_index': 125,
  'prob': 0.5003323640979196},
 {'score': 5.229421854019165,
  'text': ' 1960 के दशक के मध्य तक पारंपरिक बीजों का प्रयोग किया जाता था',
  'start_index': 122,
  'end_index': 137,
  'prob': 0.4982003542330574},
 {'score': 4.565392732620239,
  'text'

In [170]:
eval_ans_id_sel = sorted(eval_ans_id, key=lambda x: x["score"], reverse=True)[:4]

In [171]:
eval_ans_id_sel

[{'score': 13.138822555541992,
  'text': ' 1960',
  'start_index': 122,
  'end_index': 122,
  'prob': 0.8369724266622645},
 {'score': 11.97038221359253,
  'text': ' 1960 के दशक',
  'start_index': 122,
  'end_index': 124,
  'prob': 0.6034223452136326},
 {'score': 10.8192138671875,
  'text': ' 1960 के दशक के मध्य',
  'start_index': 122,
  'end_index': 126,
  'prob': 0.531393499340414},
 {'score': 9.912975311279297,
  'text': ' 1960 के दशक के मध्य तक',
  'start_index': 122,
  'end_index': 127,
  'prob': 0.5115375728067632}]

In [173]:
eval_df = pd.DataFrame(eval_ans_id_sel)

In [174]:
eval_df

,score,text,start_index,end_index,prob
0,13.138823,1960,122,122,0.836972
1,11.970382,1960 के दशक,122,124,0.603422
2,10.819214,1960 के दशक के मध्य,122,126,0.531393
3,9.912975,1960 के दशक के मध्य तक,122,127,0.511538


In [177]:
_get_jem(eval_df)

[1.2732690977965981, 1.5878808222904226, 1.602433423161615, 1.4661002647397268]

In [175]:
list(itertools.product(eval_df['text'].values,eval_df['text'].values))

[(' 1960', ' 1960'),
 (' 1960', ' 1960 के दशक'),
 (' 1960', ' 1960 के दशक के मध्य'),
 (' 1960', ' 1960 के दशक के मध्य तक'),
 (' 1960 के दशक', ' 1960'),
 (' 1960 के दशक', ' 1960 के दशक'),
 (' 1960 के दशक', ' 1960 के दशक के मध्य'),
 (' 1960 के दशक', ' 1960 के दशक के मध्य तक'),
 (' 1960 के दशक के मध्य', ' 1960'),
 (' 1960 के दशक के मध्य', ' 1960 के दशक'),
 (' 1960 के दशक के मध्य', ' 1960 के दशक के मध्य'),
 (' 1960 के दशक के मध्य', ' 1960 के दशक के मध्य तक'),
 (' 1960 के दशक के मध्य तक', ' 1960'),
 (' 1960 के दशक के मध्य तक', ' 1960 के दशक'),
 (' 1960 के दशक के मध्य तक', ' 1960 के दशक के मध्य'),
 (' 1960 के दशक के मध्य तक', ' 1960 के दशक के मध्य तक')]

In [176]:
eval_df['prob'].tolist() *4

[0.8369724266622645,
 0.6034223452136326,
 0.531393499340414,
 0.5115375728067632,
 0.8369724266622645,
 0.6034223452136326,
 0.531393499340414,
 0.5115375728067632,
 0.8369724266622645,
 0.6034223452136326,
 0.531393499340414,
 0.5115375728067632,
 0.8369724266622645,
 0.6034223452136326,
 0.531393499340414,
 0.5115375728067632]

In [300]:
prob_score

[0.9676578351816592,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8836736327753607,
 0.0,
 0.868888718473327,
 0.0,
 0.0,
 0.8347426752550527,
 0.0,
 0.0,
 0.8836736327753607,
 0.0,
 0.868888718473327]

In [180]:
combination_stor =[]
for i in jacc_combination:
    combination_stor.append(jaccard(i[0],i[1]))
    print(jaccard(i[0],i[1]), "///",i[0],i[1])

0.0 ///  1,250  12.6%
0.0 ///  1,250  50
0.0 ///  1,250  12.6%
0.0 ///  12.6%  50
1.0 ///  12.6%  12.6%
0.0 ///  50  12.6%


In [235]:
for idx0 ,idx1 in jacc_combination:
    print(idx0)
    print('daddffsdf',idx1)

 1,250
daddffsdf  12.6%
 1,250
daddffsdf  50
 1,250
daddffsdf  12.6%
 12.6%
daddffsdf  50
 12.6%
daddffsdf  12.6%
 50
daddffsdf  12.6%


In [181]:
combination_stor

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [222]:
_get_je(1,2)

0.0

In [227]:
eval_df['prob'][1] * _get_je(0,1) +eval_df['prob'][1] * _get_je(1,1)+eval_df['prob'][1] * _get_je(1,2)+eval_df['prob'][1] * _get_je(1,3)

1.7673472655507214

In [231]:
_get_je(1,3)

1.0

In [232]:
jem =  []
for i in range(4):
    je = 0.0
    for j in range(4):
        je += eval_df['prob'][j]*_get_je(i,j)
    jem.append(je)
        

In [240]:
def _get_jem_score(eval_df):
    
    combination_jcc =[]
    for str1, str2 in list(itertools.combinations(eval_df['text'].values,2)):
        combination_jcc.append(jaccard(str1,str2))
    
    jem =  []
    for i in range(4):
        je = 0.0
        for j in range(4):
            je += eval_df['prob'][j]*_get_je(i,j)
        jem.append(je)
        
    return jem

In [241]:
# eval_df['jem_score'] = _get_jem_score(eval_df)

In [110]:

jaccard_exp_max = []

for i in range(len(eval_ans_id_sel)-1):
    print(eval_ans_id_sel[i]['prob'] , jaccard(i['text'],) )
    
    jaccard_expectation_logit += logit * jaccard_val
    
    
        

0.9676578351816592
0.8836736327753607
0.8347426752550527
0.868888718473327


In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return safe_div(float(len(c)) , (len(a) + len(b) - len(c)))


#### Playing with logits 

In [25]:
len(start_logits) , len(end_logits)

(19, 19)

In [27]:
valid_set.head()

,id,context,question,answer_text,answer_start,language,kfold,answers
0,2644589b4,हाइड्रोजन (उदजन) (अंग्रेज़ी:Hydrogen) एक रासाय...,हाइड्रोजन का परमाणु द्रव्यमान क्या है?,1.008,688,hindi,4,"{'answer_start': [688], 'text': ['1.008']}"
1,2b36b228a,"नाभिक, परमाणु के मध्य स्थित धनात्मक वैद्युत आव...",नाभिक की खोज सबसे पहले किस साल में की गयी थी?,सन १९१२,2006,hindi,4,"{'answer_start': [2006], 'text': ['सन १९१२']}"
2,006bd6b9a,विश्व पर्यावरण दिवस पर्यावरण की सुरक्षा और संर...,पहला विश्व पर्यावरण दिवस किस वर्ष आयोजित किया ...,1974,347,hindi,4,"{'answer_start': [347], 'text': ['1974']}"
3,3b5cc5941,"""विश्व व्यापी वितान,"" ""W3,"" और ""WWW"" यहाँ पुनर...",विश्व व्यापी वेब(डब्ल्यूडब्ल्यूडब्ल्यू) को किस...,1992,733,hindi,3,"{'answer_start': [733], 'text': ['1992']}"
4,839bdc1fc,"संयुक्त राष्ट्र () एक अंतरराष्ट्रीय संगठन है, ...",संयुक्त राष्ट्र संगठन किस साल में स्थापित हुआ था?,1945,1442,hindi,0,"{'answer_start': [1442], 'text': ['1945']}"


In [193]:
len(features_per_example[40])

20

In [264]:
start_logits_stack = np.vstack(start_logits)
end_logits_stack = np.vstack(end_logits)
    

In [265]:
example_id_to_index = {k: i for i, k in enumerate(valid_set['id'])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(valid_features):
    features_per_example[example_id_to_index[feature['example_id']]].append(i)

In [266]:
features_per_example[example_id_to_index['01a91f15a']]

[7004,
 7005,
 7006,
 7007,
 7008,
 7009,
 7010,
 7011,
 7012,
 7013,
 7014,
 7015,
 7016,
 7017,
 7018,
 7019,
 7020,
 7021,
 7022,
 7023]

In [267]:
one_ex_start = start_logits_stack[7004]
one_ex_end = end_logits_stack[7004]


In [272]:
F.log_softmax(torch.tensor(one_ex_start),dim=0)

tensor([ -2.8060, -13.4118, -13.8145, -12.8922, -13.0544, -12.8533, -13.8871,
        -13.4560, -13.1360, -13.2474, -14.0932, -13.9523, -13.7459, -14.4704,
        -10.9259, -11.9724, -13.6982, -13.7314, -13.0862, -12.9611, -11.7383,
        -14.0180, -11.4882, -13.9197, -12.8504, -13.7304, -13.9874, -13.7814,
        -13.3807, -12.8150, -13.1617, -13.3431, -12.9135, -14.0515, -12.9621,
        -13.8660, -13.3897, -13.2988, -13.8603, -13.0412, -14.1184, -13.8206,
        -14.3310, -14.1228, -13.9025, -14.0818, -13.7641, -13.2949, -13.8092,
        -13.2079, -13.5191, -14.2351, -13.2047, -13.8517, -13.8732, -14.3409,
        -14.1767, -13.7478, -13.5750, -14.0732, -14.1460, -14.1612, -14.2690,
        -14.2323, -13.8166, -14.0207, -13.0274, -13.8527, -14.0356, -14.3078,
        -13.6503, -14.2144, -13.6807, -14.2467, -13.2614, -14.2942, -14.0147,
        -14.2260, -14.2407, -14.4804, -14.1364, -13.2735, -13.7879, -13.9881,
        -14.0061, -13.9349, -14.4586, -14.2486, -13.7198, -13.92

In [34]:
one_ex_start

array([ 4.14415121, -6.46158648, -6.86431217, -5.94204712, -6.10422945,
       -5.90314198, -6.93688107, -6.5058012 , -6.18577385, -6.29716253,
       -7.14303637, -7.00211763, -6.79571533, -7.52021313, -3.97568727,
       -5.02221346, -6.74802732, -6.78120041, -6.13597631, -6.0108943 ,
       -4.78812981, -7.06782436, -4.53795576, -6.96947718, -5.90019655,
       -6.78017282, -7.03724527, -6.83115578, -6.43047094, -5.86479712,
       -6.21153927, -6.3929143 , -5.96327782, -7.10133553, -6.01190329,
       -6.91576815, -6.43947935, -6.34858656, -6.91007233, -6.09104156,
       -7.16821003, -6.87042665, -7.38081884, -7.1725831 , -6.95233345,
       -7.13156891, -6.81390667, -6.34470654, -6.85900259, -6.25769281,
       -6.56886578, -7.28488827, -6.25446081, -6.90145397, -6.92300034,
       -7.39073133, -7.22650957, -6.79759884, -6.62475681, -7.12303352,
       -7.19579411, -7.21102667, -7.31882715, -7.28207445, -6.86635303,
       -7.07055187, -6.07718754, -6.90245819, -7.0854063 , -7.35

In [35]:
start_indexes = np.argsort(one_ex_start)[-1 : -20 - 1 : -1].tolist()
end_indexes = np.argsort(one_ex_end)[-1 : -20 - 1 : -1].tolist()

In [36]:
feature_index = 522

In [37]:
valid_features[522]['input_ids']

[0,
 3946,
 421,
 96671,
 180513,
 20700,
 7549,
 471,
 128450,
 996,
 78587,
 1293,
 32,
 2,
 2,
 77095,
 5673,
 30856,
 7186,
 58745,
 72551,
 8188,
 16866,
 1393,
 77095,
 5673,
 30856,
 7186,
 58745,
 85906,
 8761,
 82379,
 3946,
 421,
 29405,
 287,
 7494,
 13353,
 13098,
 127096,
 1964,
 44348,
 1480,
 12829,
 10386,
 1293,
 125,
 3946,
 471,
 38556,
 52496,
 71184,
 6698,
 356,
 18600,
 83603,
 287,
 967,
 141438,
 224406,
 629,
 51560,
 55082,
 1293,
 20,
 87536,
 20322,
 3241,
 171785,
 4,
 60242,
 70385,
 224640,
 4,
 224640,
 659,
 6698,
 1404,
 58047,
 1026,
 4,
 5450,
 80856,
 1223,
 58525,
 64444,
 8188,
 871,
 20488,
 9155,
 9404,
 80856,
 45120,
 4,
 28116,
 3044,
 2416,
 103880,
 4,
 78177,
 4,
 14054,
 16613,
 287,
 78177,
 4,
 1618,
 3044,
 7549,
 287,
 167510,
 287,
 10386,
 4,
 3765,
 2093,
 6,
 203507,
 1472,
 82900,
 3849,
 51560,
 460,
 125,
 4239,
 201,
 127096,
 1964,
 44348,
 1480,
 12829,
 12923,
 10386,
 471,
 9163,
 2410,
 10781,
 4415,
 18224,
 460,
 12,
 

In [38]:
min_null_score = None
sequence_ids = valid_features[522]['sequence_ids']
context_index = 1


offset_mapping = valid_features[522]['offset_mapping']
offset_mapping = [
    (o if sequence_ids[k] == context_index else None)
    for k, o in enumerate(offset_mapping)]

cls_index = valid_features[522]['input_ids'].index(tokenizer.cls_token_id)
feature_null_score = one_ex_start[cls_index] + one_ex_end[cls_index]
if min_null_score is None or min_null_score < feature_null_score:
    min_null_score = feature_null_score


In [39]:
min_null_score

7.871446132659912

In [40]:
for i,j in zip(start_indexes , end_indexes):
    print(i,j)

289 290
0 0
285 283
282 291
286 289
278 299
290 285
311 317
298 333
316 300
310 284
292 308
307 287
331 332
293 278
125 282
283 125
279 298
301 314
299 316


In [41]:
len(offset_mapping)

384

In [42]:
offset_mapping[289]

(885, 889)

In [43]:
context = valid_set[valid_set.id=="01a91f15a"]['context'].values[0]
actual_answer = valid_set[valid_set.id=="01a91f15a"]['answer_text'].values[0]

In [61]:
print(context)

अंगूठाकार|भारतीय मोर
अंगूठाकार|लाल पांडा
भारत में दुनिया के कुछ सबसे अधिक जैव विविधता वाले क्षेत्र हैं। भारत की राजनीतिक सीमाएँ इकोज़ोन के एक विस्तृत श्रृंखला को शामिल करती हैं - जिसमें रेगिस्तान, ऊंचे पहाड़, पहाड़ी इलाक़ा, उष्णकटिबंधीय और समशीतोष्ण वन, दलदली भूमि, मैदान, घास के मैदान, नदियों के आसपास के क्षेत्र, साथ ही द्वीपसमूह शामिल है। यह 4 जैव विविधता वाले प्रमुख क्षेत्र की मेजबानी करता है: हिमालय, पश्चिमी घाट, इंडो-बर्मा क्षेत्र और सुन्दालैंड (द्वीपों के निकोबार समूह सहित)।[1] इन प्रमुख क्षेत्र में कई स्थानिक प्रजातियां पाई जाती हैं।[2]
भारत का अधिकांश इकोज़ोन भाग, हिमालय की ऊपरी, इण्डोमालय क्षेत्र पर स्थित है, जो कि पियरएक्टिक इकोज़ोन का हिस्सा है; 2000 से 2500 मीटर के समोच्च को भारत-मलयान और पल्लिक्टिक क्षेत्रों के बीच की ऊँचाई सीमा माना जाता है। भारत महत्वपूर्ण जैव विविधता प्रदर्शित करता है। सत्रह विशालविविध देशों में से एक, यह सभी स्तनधारी के 7.6%, सभी पक्षियो के 12.6%, सभी सरीसृप के 6.2%, सभी उभयचरों के 4.4%, सभी मछलियों के 11.7% और सभी फूलों वाले पौधों की प्रजातियों के 6.0%

In [46]:
valid_answers = []
max_answer_length = 30
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue

        start_char = offset_mapping[start_index][0]
        end_char = offset_mapping[end_index][1]
        valid_answers.append(
            {
                "score": one_ex_start[start_index] + one_ex_end[end_index],
                "text": context[start_char: end_char],
                "start_index" : start_index,
                "end_index" : end_index
                
            }
        )


In [47]:
eval_ans = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:5]

In [48]:
eval_ans

[{'score': 12.631595611572266,
  'text': ' 12.6%',
  'start_index': 289,
  'end_index': 290},
 {'score': 8.952966690063477,
  'text': ' सभी पक्षियो के 12.6%',
  'start_index': 285,
  'end_index': 290},
 {'score': 8.336413860321045,
  'text': ' 12.6%,',
  'start_index': 289,
  'end_index': 291},
 {'score': 8.320702075958252,
  'text': ' 12.',
  'start_index': 289,
  'end_index': 289},
 {'score': 7.343181371688843,
  'text': ' 7.6%, सभी पक्षियो के 12.6%',
  'start_index': 282,
  'end_index': 290}]

In [49]:
actual_answer

'सभी पक्षियो के 12.6%'

In [50]:
import torch.nn.functional as F 

In [102]:
prob_start = F.softmax(torch.tensor(one_ex_start),dim=0).cpu().numpy()
prob_end = F.softmax(torch.tensor(one_ex_end)).cpu().numpy()

/tmp/ipykernel_59569/3357614923.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob_end = F.softmax(torch.tensor(one_ex_end)).cpu().numpy()


In [99]:
prob_start

array([6.04434765e-02, 1.49739398e-06, 1.00100105e-06, 2.51749814e-06,
       2.14059381e-06, 2.61737197e-06, 9.30932655e-07, 1.43262944e-06,
       1.97296774e-06, 1.76499912e-06, 7.57506136e-07, 8.72140356e-07,
       1.07207647e-06, 5.19493800e-07, 1.79865716e-05, 6.31608318e-06,
       1.12444030e-06, 1.08775106e-06, 2.07370406e-06, 2.35000728e-06,
       7.98195140e-06, 8.16676977e-07, 1.02508124e-05, 9.01077101e-07,
       2.62509260e-06, 1.08886939e-06, 8.42035967e-07, 1.03474699e-06,
       1.54471864e-06, 2.71968376e-06, 1.92278269e-06, 1.60383627e-06,
       2.46461327e-06, 7.89762666e-07, 2.34763735e-06, 9.50796312e-07,
       1.53086567e-06, 1.67652997e-06, 9.56227329e-07, 2.16901069e-06,
       7.38674951e-07, 9.94899123e-07, 5.97198264e-07, 7.35451723e-07,
       9.16658097e-07, 7.66242803e-07, 1.05275028e-06, 1.68304759e-06,
       1.00633009e-06, 1.83605622e-06, 1.34507120e-06, 6.57325767e-07,
       1.84199995e-06, 9.64504050e-07, 9.43944779e-07, 5.91307785e-07,
      

In [54]:
np.max(prob_start) , np.min(prob_start)

(0.8978412601259309, 4.6205285864118316e-07)

In [56]:
dumm_df = pd.DataFrame([prob_start,prob_end]).T

In [57]:
dumm_df['start_logit']=one_ex_start
dumm_df['end_logit'] = one_ex_end

In [58]:
dumm_df.columns= ['start_prob','end_prob','start_logit','end_logit']

In [59]:
dumm_df

,start_prob,end_prob,start_logit,end_logit
0,6.044348e-02,1.068537e-01,4.144151,3.727295
1,1.497394e-06,1.754317e-06,-6.461586,-7.289842
2,1.001001e-06,1.609549e-06,-6.864312,-7.375968
3,2.517498e-06,2.223595e-06,-5.942047,-7.052796
4,2.140594e-06,2.836712e-06,-6.104229,-6.809276
...,...,...,...,...
379,1.108798e-06,1.980619e-06,-6.762036,-7.168512
380,1.385473e-06,2.097403e-06,-6.539271,-7.111221
381,7.297249e-07,4.607709e-06,-7.180400,-6.324191
382,2.110434e-06,1.812780e-06,-6.118419,-7.257060


In [157]:
one_ex_start[289] , one_ex_end[290]

(6.842435836791992, 5.789159774780273)

In [162]:
prob_start[289], prob_end[290] , 0.5* (prob_start[289]+ prob_end[290])

(0.8978412601259309, 0.8399361768207232, 0.868888718473327)

In [163]:
prob_start[285], prob_end[290] , 0.5* (prob_start[285]+ prob_end[290])

(0.02267729856118877, 0.8399361768207232, 0.43130673769095595)

In [164]:
prob_start[289], prob_end[291] , 0.5* (prob_start[289]+ prob_end[291])

(0.8978412601259309, 0.011451768333975447, 0.45464651422995317)

In [168]:
 (0.868888718473327 *  jaccard(eval_ans[0]['text'],eval_ans[0]['text']) )+ (0.43130673769095595*  jaccard(eval_ans[0]['text'],eval_ans[1]['text'])) +(0.45464651422995317* jaccard(eval_ans[0]['text'],eval_ans[2]['text']) )

0.9767154028960661

In [169]:
 (0.868888718473327 *  jaccard(eval_ans[1]['text'],eval_ans[0]['text']) )+ (0.43130673769095595*  jaccard(eval_ans[1]['text'],eval_ans[1]['text'])) +(0.45464651422995317* jaccard(eval_ans[1]['text'],eval_ans[2]['text']) )

0.6485289173092877

In [170]:
 (0.868888718473327 *  jaccard(eval_ans[2]['text'],eval_ans[0]['text']) )+ (0.43130673769095595*  jaccard(eval_ans[2]['text'],eval_ans[1]['text'])) +(0.45464651422995317* jaccard(eval_ans[2]['text'],eval_ans[2]['text']) )

0.45464651422995317

In [60]:
dumm_df[dumm_df.index.isin([288,289,290])]

,start_prob,end_prob,start_logit,end_logit
288,0.000011,0.000050,-4.429681,-3.940982
289,0.897841,0.011273,6.842436,1.478266
290,0.001445,0.839936,0.410447,5.789160


In [205]:
end_indexes

[[13,
  191,
  261,
  260,
  259,
  258,
  257,
  256,
  255,
  254,
  253,
  252,
  251,
  250,
  249,
  248,
  247,
  246,
  245,
  244,
  243,
  242,
  262,
  263,
  264,
  265,
  285,
  284,
  283,
  282,
  281,
  280,
  279,
  278,
  277,
  241,
  276,
  274,
  273,
  272,
  271,
  270,
  269,
  268,
  267,
  266,
  275,
  240,
  239,
  238,
  213,
  212,
  211,
  210,
  209,
  208,
  207,
  206,
  205,
  214,
  204,
  202,
  201,
  200,
  199,
  198,
  197,
  196,
  195,
  194,
  203,
  286,
  215,
  217,
  237,
  236,
  235,
  234,
  233,
  232,
  231,
  230,
  229,
  216,
  228,
  226,
  225,
  224,
  223,
  222,
  221,
  220,
  219,
  218,
  227,
  287,
  288,
  289,
  357,
  356,
  355,
  354,
  353,
  352,
  351,
  350,
  349,
  358,
  348,
  346,
  345,
  344,
  343,
  342,
  341,
  340,
  339,
  338,
  347,
  337,
  359,
  361,
  381,
  380,
  379,
  378,
  377,
  376,
  375,
  374,
  373,
  360,
  372,
  370,
  369,
  368,
  367,
  366,
  365,
  364,
  363,
  362,
  371,


In [229]:
np.min(one_ex_start[0])

-7.637388706207275

In [211]:
dumm = pd.DataFrame(start_indexes).T

In [212]:
dumm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,140,220,83,104,320,287,317,151,186,304,128,26,266,141,137,114,333,323,220,219
1,160,356,145,80,47,77,181,153,187,279,249,22,141,274,109,377,354,312,237,251
2,153,149,198,323,119,353,272,314,39,287,130,370,124,113,61,108,28,243,137,236
3,112,190,152,209,113,316,260,111,64,317,195,70,119,350,168,301,72,223,200,137
4,145,372,181,316,344,359,74,135,175,271,122,368,205,288,201,327,348,217,242,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,141,126,368,8,229,219,293,14,150,139,19,252,342,72,235,163,187,167,38,286
380,74,128,287,13,227,49,289,286,132,27,23,244,363,101,239,123,192,154,45,282
381,167,14,300,5,14,16,46,382,14,190,29,189,359,209,237,148,14,24,42,285
382,14,88,328,14,232,14,14,293,148,14,18,0,14,41,0,14,183,153,0,0


In [213]:
dumm_e = pd.DataFrame(end_indexes).T

In [214]:
dumm_e

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,13,199,121,213,13,13,177,13,118,13,191,315,112,13,13,13,13,13,13,13
1,191,13,13,192,383,383,179,383,88,383,192,313,264,383,383,383,383,383,383,383
2,261,383,383,123,147,313,220,136,181,358,182,182,145,143,119,189,74,220,212,229
3,260,249,200,324,130,164,13,132,129,297,13,316,182,315,136,195,46,320,227,196
4,259,173,206,106,73,169,383,117,103,349,383,312,113,352,343,37,80,305,302,249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,141,123,376,10,226,71,307,373,152,82,21,251,5,216,245,160,198,166,51,289
380,17,132,380,377,228,67,59,14,14,227,32,189,15,102,241,169,33,27,49,291
381,14,127,330,15,14,27,15,382,155,15,29,250,378,210,238,164,204,157,43,283
382,15,14,367,14,232,14,14,299,148,14,18,0,14,41,0,14,14,153,0,0


In [192]:
example_id_to_index['01a91f15a']

40

In [187]:
d

,id,predicted,score,actual_ans,jaccard,sum_jaccard_batch,cum_batch_jacc
2,006bd6b9a,1974,12.196301,1974,1.0,2.5,0.833333
51,00f3b6833,अबुजा,14.801345,अबुजा,1.0,2.0,1.0
54,01a91f15a,"1,250",15.129936,सभी पक्षियो के 12.6%,0.0,0.0,0.0
46,0491db7b9,३२४ मीटर,14.476801,३२४ मीटर,1.0,4.833333,0.805556
58,0a2966553,कंप्यूटर,12.561255,कंप्यूटर,1.0,3.0,0.75
45,1473492ab,1960,13.138823,1960,1.0,3.833333,0.766667
49,229df2339,होली,12.453424,होली,1.0,1.0,0.5
22,23cbf5338,दक्षिणी गोलार्द्ध के अंटार्कटिक क्षेत्र,10.985020,140 लाख वर्ग किलोमीटर,0.0,2.0,0.5
52,23dbf4302,1832,11.680597,1769,0.0,2.0,0.666667
60,2528af932,चार्ल्स लुई अल्फोंस लैवेरन,12.119678,चार्ल्स लुई अल्फोंस लैवेरन,1.0,1.0,0.5


In [174]:
cc = 0
for i in ans_pred:
    bb = 0
    for idx,k in enumerate(i.keys()):
        bb += i[k][-3]
    score = 0 if bb == 0 else bb/(idx+1)
    print("batch score is ",score)
    cc += score

batch score is  0.875
batch score is  0.4791666666666667
batch score is  0
batch score is  0.6944444444444443
batch score is  0.5
batch score is  0.8
batch score is  0.6666666666666666
batch score is  0.5
batch score is  0.5
batch score is  0
batch score is  0.5
batch score is  0.738095238095238
batch score is  0.5
batch score is  0.6666666666666666
batch score is  0
batch score is  0.75
batch score is  0.3333333333333333
batch score is  0.25
batch score is  0


In [171]:
dfs =[]
for i in range(len(ans_pred)):
    dfs.append(pd.DataFrame(ans_pred[i]).T)
    
cc = pd.concat(dfs)
cc.reset_index(inplace=True)

In [175]:
cc.columns = ['id',"predicted","score","actual_ans","jaccard","sum_jaccard_batch","cum_batch_jacc"]

In [177]:
cc[cc.id=="01a91f15a"]

,id,predicted,score,actual_ans,jaccard,sum_jaccard_batch,cum_batch_jacc
54,01a91f15a,"1,250",15.129936,सभी पक्षियो के 12.6%,0.0,0.0,0.0
55,01a91f15a,सांप,-9.407011,सभी पक्षियो के 12.6%,0.0,0.0,0.0


In [164]:
cc.score = cc.score.astype('float')

In [167]:
d = cc.loc[cc.groupby('id')['score'].idxmax()]

In [172]:
d[d.jaccard==1].shape , d[d.jaccard!=1].shape , d.shape

((32, 7), (18, 7), (50, 7))

In [173]:
d[d.jaccard!=1]

,id,predicted,score,actual_ans,jaccard,sum_jaccard_batch,cum_batch_jacc
54,01a91f15a,"1,250",15.129936,सभी पक्षियो के 12.6%,0.0,0.0,0.0
22,23cbf5338,दक्षिणी गोलार्द्ध के अंटार्कटिक क्षेत्र,10.985020,140 लाख वर्ग किलोमीटर,0.0,2.0,0.5
52,23dbf4302,1832,11.680597,1769,0.0,2.0,0.666667
1,2b36b228a,१९१२,12.946884,सन १९१२,0.5,1.5,0.75
10,2bfd1f74c,स्विटजरलैंड के जेनेवा,13.239895,जेनेवा,0.333333,3.833333,0.547619
7,41b127607,फ़ेलिडाए,13.727931,फ़ेलिडाए कुल,0.5,1.5,0.375
36,46144ebc7,VAT,5.095928,वस्तु और सेवा कर,0.0,0.0,0.0
42,73b394d20,1974,12.216961,5 जून 1974,0.333333,1.333333,0.666667
5,839bdc1fc,१९४५,12.908390,1945,0.0,0.0,0.0
65,944251e8e,षड्दर्शन,10.796417,छः,0.0,1.0,0.25


In [90]:
ans_pred

[OrderedDict([('2644589b4', (' 1.008', 12.768225, '1.008', 1.0, 1.0, 1.0)),
              ('2b36b228a', (' १९१२', 12.946884, 'सन १९१२', 0.5, 1.5, 0.75)),
              ('006bd6b9a',
               (' 1974', 12.196301, '1974', 1.0, 2.5, 0.8333333333333334)),
              ('3b5cc5941', (' 1992', 12.766145, '1992', 1.0, 3.5, 0.875))]),
 OrderedDict([('3b5cc5941',
               ('wàn wéi wǎng', -11.206213, '1992', 0.0, 0.0, 0.0)),
              ('839bdc1fc', (' १९४५', 12.90839, '1945', 0.0, 0.0, 0.0)),
              ('40f4e32da',
               (' मीटर प्रति सेकेण्ड2',
                13.8907585,
                'मीटर प्रति सेकेण्ड2',
                1.0,
                1.0,
                0.3333333333333333)),
              ('41b127607',
               (' फ़ेलिडाए', 13.727931, 'फ़ेलिडाए कुल', 0.5, 1.5, 0.375)),
              ('557cf465b',
               (' जार्ज ओह्म', 11.041241, 'जार्ज ओह्म', 1.0, 2.5, 0.5)),
              ('291615e22',
               ('लैमार्क और ट्रविरेनस',
       

In [79]:
cc = collections.OrderedDict()

In [81]:
example_id_to_index

{'2644589b4': 0,
 '2b36b228a': 1,
 '006bd6b9a': 2,
 '3b5cc5941': 3,
 '839bdc1fc': 4,
 '40f4e32da': 5,
 '41b127607': 6,
 '557cf465b': 7,
 '291615e22': 8,
 '2bfd1f74c': 9,
 'f88f5c088': 10,
 '96069c1b9': 11,
 '4eb7216bf': 12,
 '7b6a3ba94': 13,
 'bd0229b4c': 14,
 '7128ae116': 15,
 '3acaa3b86': 16,
 '9e966c3f9': 17,
 '23cbf5338': 18,
 '56f522624': 19,
 '28045a331': 20,
 '53fcc326b': 21,
 'ff31c3eba': 22,
 'c9c773261': 23,
 '5aeacd81a': 24,
 'f66bb1e4b': 25,
 '67e6695dc': 26,
 '46144ebc7': 27,
 '934a3199c': 28,
 '642ff0529': 29,
 'e9d3a3082': 30,
 '73b394d20': 31,
 'ebfa0db53': 32,
 'ce689632e': 33,
 '1473492ab': 34,
 '0491db7b9': 35,
 'c196ba459': 36,
 '229df2339': 37,
 '00f3b6833': 38,
 '23dbf4302': 39,
 '01a91f15a': 40,
 '9031feef1': 41,
 'af30436d6': 42,
 '0a2966553': 43,
 '2528af932': 44,
 'b434bae7c': 45,
 'b743d8294': 46,
 'af2573e66': 47,
 'ea9f54b7b': 48,
 '944251e8e': 49}

In [84]:
example_id_to_index

{'2644589b4': 0,
 '2b36b228a': 1,
 '006bd6b9a': 2,
 '3b5cc5941': 3,
 '839bdc1fc': 4,
 '40f4e32da': 5,
 '41b127607': 6,
 '557cf465b': 7,
 '291615e22': 8,
 '2bfd1f74c': 9,
 'f88f5c088': 10,
 '96069c1b9': 11,
 '4eb7216bf': 12,
 '7b6a3ba94': 13,
 'bd0229b4c': 14,
 '7128ae116': 15,
 '3acaa3b86': 16,
 '9e966c3f9': 17,
 '23cbf5338': 18,
 '56f522624': 19,
 '28045a331': 20,
 '53fcc326b': 21,
 'ff31c3eba': 22,
 'c9c773261': 23,
 '5aeacd81a': 24,
 'f66bb1e4b': 25,
 '67e6695dc': 26,
 '46144ebc7': 27,
 '934a3199c': 28,
 '642ff0529': 29,
 'e9d3a3082': 30,
 '73b394d20': 31,
 'ebfa0db53': 32,
 'ce689632e': 33,
 '1473492ab': 34,
 '0491db7b9': 35,
 'c196ba459': 36,
 '229df2339': 37,
 '00f3b6833': 38,
 '23dbf4302': 39,
 '01a91f15a': 40,
 '9031feef1': 41,
 'af30436d6': 42,
 '0a2966553': 43,
 '2528af932': 44,
 'b434bae7c': 45,
 'b743d8294': 46,
 'af2573e66': 47,
 'ea9f54b7b': 48,
 '944251e8e': 49}

In [83]:
features_per_example

defaultdict(list,
            {0: [0, 1, 2, 3, 4, 5, 6],
             1: [7, 8, 9],
             2: [10],
             3: [11,
              12,
              13,
              14,
              15,
              16,
              17,
              18,
              19,
              20,
              21,
              22,
              23,
              24,
              25,
              26,
              27,
              28,
              29,
              30,
              31,
              32,
              33,
              34,
              35,
              36,
              37,
              38,
              39],
             4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
             5: [50, 51, 52, 53, 54],
             6: [55],
             7: [56, 57],
             8: [58, 59, 60, 61],
             9: [62, 63],
             10: [64,
              65,
              66,
              67,
              68,
              69,
              70,
              71,
              72

In [85]:
example_id_to_index1 = {k: i for i, k in enumerate(list(dict.fromkeys(data['id'])))}
features_per_example1 = collections.defaultdict(list)
for i, feature in enumerate(data['id']):
    features_per_example1[example_id_to_index1[feature]].append(i)

In [87]:
example_id_to_index1

{'944251e8e': 0}

In [86]:
features_per_example1

defaultdict(list,
            {0: [0,
              1,
              2,
              3,
              4,
              5,
              6,
              7,
              8,
              9,
              10,
              11,
              12,
              13,
              14,
              15,
              16,
              17,
              18]})

In [16]:
def _get_qa_pred(data, start_logits0,end_logits0,tokenizer,n_best_size = 20,max_answer_length = 30):

    example_id_to_index = {k: i for i, k in enumerate(list(dict.fromkeys(data['id'])))}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(data['id']):
        features_per_example[example_id_to_index[feature]].append(i)

    predictions = collections.OrderedDict()
    
    answer_prediction = []
    
    
    

    start_logits_stack = np.vstack(start_logits0.detach().cpu().numpy())
    end_logits_stack = np.vstack(end_logits0.detach().cpu().numpy())

    
    jaccard_batch = 0.0
    


    for k,v in example_id_to_index.items():

        feature_indices = features_per_example[v]


        min_null_score = None

        valid_answers = []
        
        context = data['context'][feature_indices[0]]


        for feature_index in feature_indices:
            start_logits = start_logits_stack[feature_index]
            end_logits   = end_logits_stack[feature_index]



            sequence_ids = [ int(i[feature_index].detach().cpu().numpy()) for i in  data['sequence_ids']]
            context_index = 1


            offset_mapping = [ tuple(i) for i in data['offset_mapping'][feature_index].detach().cpu().numpy()]
            offset_mapping = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(offset_mapping)]

            cls_index = data['input_ids'][feature_index].tolist().index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        
        predictions[k] = best_answer["text"]

    return predictions


In [19]:
def _get_jaccard_score(data, start_logits0,end_logits0,tokenizer,n_best_size = 20,max_answer_length = 30):

    example_id_to_index = {k: i for i, k in enumerate(list(dict.fromkeys(data['id'])))}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(data['id']):
        features_per_example[example_id_to_index[feature]].append(i)

    predictions = collections.OrderedDict()
    
    answer_prediction = []
    
    
    

    start_logits_stack = np.vstack(start_logits0.detach().cpu().numpy())
    end_logits_stack = np.vstack(end_logits0.detach().cpu().numpy())

    
    jaccard_batch = 0.0
    count_batch = 0


    for k,v in example_id_to_index.items():

        feature_indices = features_per_example[v]


        min_null_score = None

        valid_answers = []
        count_batch +=1
        
        context = data['context'][feature_indices[0]]
        true_answer = data['answer_text'][feature_indices[0]]


        for feature_index in feature_indices:
            start_logits = start_logits_stack[feature_index]
            end_logits   = end_logits_stack[feature_index]



            sequence_ids = [ int(i[feature_index].detach().cpu().numpy()) for i in  data['sequence_ids']]
            context_index = 1


            offset_mapping = [ tuple(i) for i in data['offset_mapping'][feature_index].detach().cpu().numpy()]
            offset_mapping = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(offset_mapping)]

            cls_index = data['input_ids'][feature_index].tolist().index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

#         print(jaccard(true_answer,best_answer["text"]), "/////",true_answer,"////",best_answer["text"])
        jaccard_batch += jaccard(true_answer,best_answer["text"])
        
        predictions[k] = best_answer["text"] ,best_answer["score"],true_answer ,jaccard(true_answer,best_answer["text"]),jaccard_batch,jaccard_batch/count_batch

            
    return jaccard_batch,predictions


def safe_div(x,y):
    if y == 0:
        return 1
    return x / y

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return safe_div(float(len(c)) , (len(a) + len(b) - len(c)))



        

In [82]:
_get_jaccard_score(data, outputs_start,outputs_end,tokenizer,n_best_size = 20,max_answer_length = 30)


0.0 ///// रोज़ाली हेल ////  एरिक योर्की
0.6666666666666666 ///// सन ३३० ईसापूर्व ////  ३३० ईसापूर्व
1.0 ///// नेताजी ////  नेताजी


1.6666666666666665

In [83]:
new_get_jaccard_score(data, outputs_start,outputs_end,tokenizer,n_best_size = 20,max_answer_length = 30)


0.0 ///// रोज़ाली हेल ////  क्रिस वेईत्ज़
0.0 ///// सन ३३० ईसापूर्व ////  (1979)
0.0 ///// नेताजी ////  द आवर


0.0

In [74]:
example_id_to_index = {k: i for i, k in enumerate(data["id"])}

In [201]:

example_id_to_index = {k: i for i, k in enumerate(data["id"])}

start_logits_all = np.vstack(outputs_start.cpu().numpy().tolist())
end_logits_all = np.vstack(outputs_end.cpu().numpy().tolist())



    


In [207]:
example_id_to_index

{'6613': 0, '6614': 1, '6615': 2}

In [209]:
for feature_indices in example_id_to_index.values():
    

0
1
2


In [152]:
start_logits = start_logits_all[0]
end_logits = end_logits_all[0]

In [174]:
context = data['context'][0]

In [171]:
n_best_size = 20
max_answer_length = 30

In [154]:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

In [159]:
cls_index = data['input_ids'][0].tolist().index(tokenizer.cls_token_id)
min_null_score=None
feature_null_score = start_logits[cls_index] + end_logits[cls_index]
if min_null_score is None or min_null_score < feature_null_score:
    min_null_score = feature_null_score


In [160]:
feature_null_score

1.8939080834388733

In [162]:
sequence_ids = [ int(i[0].detach().cpu().numpy()) for i in  data['sequence_ids']]
context_index = 1

In [164]:
offset_mapping = [ tuple(i) for i in data['offset_mapping'][0].detach().cpu().numpy()]
offset_mapping = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(offset_mapping)]

In [176]:
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue

        start_char = offset_mapping[start_index][0]
        end_char = offset_mapping[end_index][1]
        valid_answers.append(
            {
                "score": start_logits[start_index] + end_logits[end_index],
                "text": context[start_char: end_char]
            }
        )


In [180]:
if len(valid_answers) > 0:
    best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
else:
    best_answer = {"text": "", "score": 0.0}

In [182]:
jaccard(data['answer_text'][0],best_answer["text"])

1.0

In [126]:
jaccard_score =  new_get_jaccard_score(data, out,out_end,tokenizer,n_best_size = 20,max_answer_length = 30)
             

Outside loop
{'6613': 0, '6614': 1, '6615': 2}
//// defaultdict(<class 'list'>, {0: [0], 1: [1], 2: [2]})
inside loop 1 दबाव की शर्तें
inside loop 2 0
inside loop 1 दबाव की शर्तें
inside loop 2 1
inside loop 1 नियम-निष्ठता
inside loop 2 2


In [71]:
c = []
a = np.array([0, 1])
b = np.array([2, 3])
# np.concatenate((a, b))

In [72]:
c.append(a)
c.append(b)

In [77]:
c

[array([0, 1]), array([2, 3])]

In [74]:
np.vstack(c)

array([[0, 1],
       [2, 3]])

In [121]:
a = "hello world"
b = "world . !"

jaccard(a,b)

0.25

In [64]:
def new_get_jaccard_score(data, outputs_start,outputs_end,tokenizer,n_best_size = 20,max_answer_length = 30):
    
    example_id_to_index = {k: i for i, k in enumerate(data["id"])}
    outputs_start=outputs_start.cpu().numpy().tolist()
    outputs_end = outputs_end.cpu().numpy().tolist()
    
    start_logits_stack = np.vstack(outputs_start)
    end_logits_stack = np.vstack(outputs_end)
    
    jaccard_batch = 0.0
    count=0
    
    for feature_index in example_id_to_index.values():

        min_null_score = None

        valid_answers = []
        count +=1
        context = data['context'][feature_index]
        true_answer = data['answer_text'][feature_index]
        

        
            
        start_logits = start_logits_stack[feature_index]
        end_logits   = end_logits_stack[feature_index]



        sequence_ids = [ int(i[feature_index].detach().cpu().numpy()) for i in  data['sequence_ids']]
        context_index = 1


        offset_mapping = [ tuple(i) for i in data['offset_mapping'][feature_index].detach().cpu().numpy()]
        offset_mapping = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(offset_mapping)]

        cls_index = data['input_ids'][feature_index].tolist().index(tokenizer.cls_token_id)
        feature_null_score = start_logits[cls_index] + end_logits[cls_index]
        if min_null_score is None or min_null_score < feature_null_score:
            min_null_score = feature_null_score

        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue

                start_char = offset_mapping[start_index][0]
                end_char = offset_mapping[end_index][1]
                valid_answers.append(
                    {
                        "score": start_logits[start_index] + end_logits[end_index],
                        "text": context[start_char: end_char]
                    }
                )
        
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
            
        print(jaccard(true_answer,best_answer["text"]), "/////",true_answer,"////",best_answer["text"])

        jaccard_batch += jaccard(true_answer,best_answer["text"])
            
    return jaccard_batch/count
        

def safe_div(x,y):
    if y == 0:
        return 1
    return x / y

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return safe_div(float(len(c)) , (len(a) + len(b) - len(c)))




In [109]:
## inference

base_model_path = 'output/xlm_roberta_v1/'

start_logits0, end_logits0,true_vals,context_val ,out, out_end, jacc = get_predictions('checkpoint-fold-0/pytorch_model.bin',tokenizer)



Some weights of the model checkpoint at xlm-roberta/deepset/xlm-roberta-large-squad2/ were not used when initializing XLMRobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [91]:
np.vstack(jacc.cpu().numpy().tolist())

array([[ 1.6066916 , -5.85141659, -6.44851017, ..., -7.41848135,
        -7.41848135, -7.41848135],
       [ 1.32015908, -5.39860487, -5.81058645, ..., -6.88944101,
        -6.88944101, -6.88944101],
       [ 0.23241669, -5.82939243, -7.08863974, ..., -7.35943365,
        -7.35943365, -7.35943365]])

In [104]:
jacc

0.0

In [ ]:
true_vals

In [24]:
start_logits0.shape , end_logits0.shape 

((16, 384), (16, 384))

In [24]:
start_logits0.shape , end_logits0.shape 

((16, 384), (16, 384))

In [36]:
set(true_vals[0])

{'नेताजी', 'सन ३३० ईसापूर्व'}

In [27]:
start_logits0

array([[ 2.76417065, -5.70755243, -6.50486326, ..., -6.20636654,
        -5.79955721, -8.11454582],
       [ 6.41001558, -5.36787987, -5.91266584, ..., -6.83023214,
        -7.48882914, -7.85708284],
       [ 6.18761826, -5.66351175, -6.27413845, ..., -6.13501978,
        -6.57302141, -8.11645794],
       ...,
       [ 6.28268862, -5.73644352, -7.15311575, ..., -5.87167549,
        -6.58301497, -8.0095005 ],
       [ 6.46362734, -6.35442686, -7.01620436, ..., -7.33529711,
        -7.84131193, -7.95865107],
       [ 6.54774237, -6.60537291, -7.52784538, ..., -7.73268843,
        -7.73268843, -7.73268843]])

In [125]:

predictions = postprocess_qa_predictions(valid_set, valid_features, (start_logits0, end_logits0))

Post-processing 2 example predictions split into 16 features.


In [126]:
predictions

OrderedDict([('cc4c69225', ' ३३० ईसापूर्व'), ('ee569fe45', ' नेताजी')])

In [127]:
valid_set

,id,context,question,answer_text,answer_start,language,kfold,answers
0,cc4c69225,हख़ामनी वंश या अजमीढ़ साम्राज्य(अंग्रेज़ी तथा ...,आचमेनिड साम्राज्य का अंत किस साल में हुआ था?,सन ३३० ईसापूर्व,522,hindi,0,"{'answer_start': [522], 'text': ['सन ३३० ईसापू..."
1,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,0,"{'answer_start': [2017], 'text': ['नेताजी']}"


In [46]:
start_logits0, end_logits0,true_vals,prediction_vals,context_val 

16

In [116]:
example_id_to_index

{'ee569fe45': 0, 'cc4c69225': 1}

In [139]:
features_per_example

defaultdict(list,
            {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [10, 11, 12, 13, 14, 15]})

In [137]:
example_id_to_index

{'cc4c69225': 0, 'ee569fe45': 1}

In [27]:
for k,v in example_id_to_index.items():
    print (f'key is {k} and value is {v}')

key is cc4c69225 and value is 0
key is ee569fe45 and value is 1


In [46]:
features_per_example[1]

[10, 11, 12, 13, 14, 15]

In [47]:
data['answer_text'] [ features_per_example[1][0]]

'नेताजी'

In [15]:
cc = np.stack([np.stack([d for d in d_]) for d_ in data['offset_mapping']])


In [15]:
data['offset_mapping'].detach()

tensor([[[   0,    0],
         [   0,    1],
         [   1,    2],
         ...,
         [1274, 1276],
         [1276, 1277],
         [   0,    0]],

        [[   0,    0],
         [   0,    1],
         [   1,    2],
         ...,
         [2137, 2139],
         [2139, 2141],
         [   0,    0]],

        [[   0,    0],
         [   0,    1],
         [   1,    2],
         ...,
         [2900, 2901],
         [2901, 2904],
         [   0,    0]],

        ...,

        [[   0,    0],
         [   0,    2],
         [   2,    4],
         ...,
         [3772, 3774],
         [3774, 3776],
         [   0,    0]],

        [[   0,    0],
         [   0,    2],
         [   2,    4],
         ...,
         [4575, 4576],
         [4576, 4578],
         [   0,    0]],

        [[   0,    0],
         [   0,    2],
         [   2,    4],
         ...,
         [   0,    0],
         [   0,    0],
         [   0,    0]]])

In [149]:
valid_features[1]['offset_mapping']

[(0, 0),
 (0, 1),
 (1, 2),
 (2, 5),
 (5, 7),
 (7, 17),
 (17, 20),
 (20, 24),
 (24, 28),
 (28, 32),
 (32, 36),
 (36, 40),
 (40, 43),
 (43, 44),
 (0, 0),
 (0, 0),
 (802, 806),
 (806, 809),
 (809, 812),
 (812, 820),
 (820, 825),
 (825, 830),
 (830, 833),
 (833, 834),
 (834, 837),
 (837, 838),
 (838, 841),
 (841, 844),
 (844, 851),
 (851, 854),
 (854, 856),
 (856, 858),
 (858, 863),
 (863, 867),
 (867, 869),
 (869, 874),
 (874, 879),
 (879, 883),
 (883, 886),
 (886, 895),
 (895, 898),
 (898, 903),
 (903, 907),
 (907, 912),
 (912, 915),
 (915, 916),
 (916, 923),
 (923, 926),
 (926, 932),
 (932, 940),
 (940, 945),
 (945, 948),
 (948, 951),
 (951, 961),
 (961, 964),
 (964, 969),
 (969, 970),
 (970, 975),
 (975, 980),
 (980, 984),
 (984, 987),
 (987, 989),
 (989, 992),
 (992, 996),
 (996, 1002),
 (1002, 1006),
 (1006, 1007),
 (1007, 1010),
 (1010, 1014),
 (1014, 1016),
 (1016, 1023),
 (1023, 1028),
 (1028, 1031),
 (1031, 1036),
 (1036, 1039),
 (1039, 1046),
 (1046, 1054),
 (1054, 1059),
 (1059

In [ ]:
[[tuple(j.detach().cpu().numpy())  for j in i] for i in data['offset_mapping']]

In [14]:
# cc=[]
[ [ d.detach().cpu().numpy() for d in i]for i in data['offset_mapping']]

[[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2])],
 [array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]),
  array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4])],
 [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4]),
  array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7])],
 [array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7]),
  array([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 10, 10, 10, 10, 10, 10])],
 [array([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 10, 10, 10, 10, 10, 10]),
  array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 15, 15, 15, 15, 15])],
 [array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 15, 15, 15, 15, 15]),
  array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21])],
 [array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21]),
  array([24,

In [91]:
[ i for i in data['offset_mapping']]

[(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])),
 (tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2])),
 (tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]),
  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4])),
 (tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4]),
  tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7])),
 (tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7]),
  tensor([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 10, 10, 10, 10, 10, 10])),
 (tensor([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 10, 10, 10, 10, 10, 10]),
  tensor([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 15, 15, 15, 15, 15])),
 (tensor([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 15, 15, 15, 15, 15]),
  tensor([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21])),
 (tensor([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21]

In [18]:
# ## orignal developed
# def _get_jaccard_score(data, start_logits,end_logits,n_best_size = 20,max_answer_length = 30):

#     example_id_to_index = {k: i for i, k in enumerate(list(dict.fromkeys(data['id'])))}
#     features_per_example = collections.defaultdict(list)
#     for i, feature in enumerate(data['id']):
#         features_per_example[example_id_to_index[feature]].append(i)

#     predictions = collections.OrderedDict()

    
    
#     jaccard_batch = 0.0
#     count=0


#     for k,v in example_id_to_index.items():

#             feature_indices = features_per_example[v]


#             min_null_score = None

#             valid_answers = []
#             count +=1
#             context = data['context'][feature_indices[0]]
#             true_answer = data['answer_text'][feature_indices[0]]


#             for feature_index in feature_indices:
#                 start_logits = start_logits0[feature_index]
#                 end_logits = end_logits0[feature_index]



#                 sequence_ids = [ int(i[feature_index].detach().cpu().numpy()) for i in  data['sequence_ids']]
#                 context_index = 1


#                 offset_mapping = [ tuple(i) for i in data['offset_mapping'][feature_index].detach().cpu().numpy()]
#                 offset_mapping = [
#                     (o if sequence_ids[k] == context_index else None)
#                     for k, o in enumerate(offset_mapping)]

#                 cls_index = data['input_ids'][feature_index].tolist().index(tokenizer.cls_token_id)
#                 feature_null_score = start_logits[cls_index] + end_logits[cls_index]
#                 if min_null_score is None or min_null_score < feature_null_score:
#                     min_null_score = feature_null_score

#                 start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
#                 end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
#                 for start_index in start_indexes:
#                     for end_index in end_indexes:
#                         if (
#                             start_index >= len(offset_mapping)
#                             or end_index >= len(offset_mapping)
#                             or offset_mapping[start_index] is None
#                             or offset_mapping[end_index] is None
#                         ):
#                             continue
#                         # Don't consider answers with a length that is either < 0 or > max_answer_length.
#                         if end_index < start_index or end_index - start_index + 1 > max_answer_length:
#                             continue

#                         start_char = offset_mapping[start_index][0]
#                         end_char = offset_mapping[end_index][1]
#                         valid_answers.append(
#                             {
#                                 "score": start_logits[start_index] + end_logits[end_index],
#                                 "text": context[start_char: end_char]
#                             }
#                         )

#             if len(valid_answers) > 0:
#                 best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
#             else:
#                 best_answer = {"text": "", "score": 0.0}




#             jaccard_batch += jaccard(true_answer,best_answer["text"])
            
#             return jaccard_batch/count
        
        

In [145]:
count

2

In [155]:
cc = " aas sdasd"
cc.lower().split()

['aas', 'sdasd', '.']

In [146]:
jaccard_batch/count

0.8333333333333333

In [141]:
predictions.get('ee569fe45').strip()

'नेताजी'

In [131]:
true_answer

'नेताजी'

In [148]:
get_jaccard_score(true_answer,"")

0.0

In [152]:
### getting a jaccard score 

def safe_div(x,y):
    if y == 0:
        return 1
    return x / y

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    score=0.0
    c = a.intersection(b)
    return safe_div(float(len(c)) , (len(a) + len(b) - len(c)))





In [ ]:
epoch_jaccard = 0.0
epoch_jaccard += get_jaccard_score(data['answer_text'],prediction_strings)

In [124]:
valid_answers

[{'score': 1.0864535570144653, 'text': ' बॉम्बे डाइंग'},
 {'score': -3.078393280506134, 'text': ' बॉ'},
 {'score': -4.0508792996406555,
  'text': ' बॉम्बे डाइंग के एक विज्ञापन में दिखाई दीं जिसमें वह करन कपूर'},
 {'score': -4.095130741596222, 'text': ' बॉम्बे'},
 {'score': -4.345264256000519, 'text': ' बॉम्बे डाइंग के एक विज्ञापन'},
 {'score': 1.1253570020198822, 'text': ' फेडरिको फेलिनी और सत्यजित राय'},
 {'score': -2.2119570672512054, 'text': ' फेडरिको फेलिनी'},
 {'score': -2.83017835021019,
  'text': ' फेडरिको फेलिनी और सत्यजित राय के फिल्म'},
 {'score': -3.837751477956772, 'text': ' फेडरिको फेलिनी और सत्यजित'},
 {'score': -4.096827119588852, 'text': ' फे'},
 {'score': -4.684332460165024, 'text': ' फेडरिको फेलिनी और सत्य'},
 {'score': -4.960113137960434, 'text': ' फेडरिको फेलिनी और सत्यजित राय के'},
 {'score': -5.428118795156479, 'text': ' फेडरिको'},
 {'score': -0.7606950402259827, 'text': ' सत्यजित राय'},
 {'score': -4.716230392456055, 'text': ' सत्यजित राय के फिल्म'},
 {'score': -

In [123]:
best_answer

{'score': 8.388649463653564, 'text': ' नेताजी'}

In [95]:

[ int(i[0].detach().cpu().numpy()) for i in  data['sequence_ids']] 

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [110]:
[  tuple(i) for i in data['offset_mapping'][0].detach().cpu().numpy()] == valid_features[0]['offset_mapping'] 

True

In [120]:
data['input_ids'][0].tolist().index(tokenizer.cls_token_id)

0

In [116]:
valid_features[0]['input_ids'].index(tokenizer.cls_token_id)

0

In [47]:
feature_indices = features_per_example[example_index]

In [145]:
features_per_example

defaultdict(list,
            {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [10, 11, 12, 13, 14, 15]})

In [40]:
## decoding the post processing

example_id_to_index = {k: i for i, k in enumerate(valid_set["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(valid_features):
#     print (f'i is {i} and feature is {feature["example_id"]}')
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

predictions = collections.OrderedDict()


for example_index, example in valid_set.head(1).iterrows():
    
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        print(context)
        
#         for feature_index in feature_indices:
#             start_logits = start_logits0[feature_index]
#             end_logits = end_logits0[feature_index]

            
            
#             sequence_ids = features[feature_index]["sequence_ids"]
#             context_index = 1

#             features[feature_index]["offset_mapping"] = [
#                 (o if sequence_ids[k] == context_index else None)
#                 for k, o in enumerate(features[feature_index]["offset_mapping"])
#             ]
#             offset_mapping = features[feature_index]["offset_mapping"]
#             cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
#             feature_null_score = start_logits[cls_index] + end_logits[cls_index]
#             if min_null_score is None or min_null_score < feature_null_score:
#                 min_null_score = feature_null_score

      

हख़ामनी वंश या अजमीढ़ साम्राज्य(अंग्रेज़ी तथा ग्रीक में एकेमेनिड, अजमीढ़ साम्राज्य (ईसापूर्व 550 - ईसापूर्व 330) प्राचीन ईरान (फ़ारस) का एक शासक वंश था। यह प्राचीन ईरान के ज्ञात इतिहास का पहला शासक वंश था जिसने आज के लगभग सम्पूर्ण ईरान पर अपनी प्रभुसत्ता हासिल की थी और इसके अलावा अपने चरमकाल में तो यह पश्मिम में यूनान से लेकर पूर्व में सिंधु नदी तक और उत्तर में कैस्पियन सागर से लेकर दक्षिण में अरब सागर तक फैल गया था। इतना बड़ा साम्राज्य इसके बाद बस सासानी शासक ही स्थापित कर पाए थे। इस वंश का पतन सिकन्दर के आक्रमण से सन ३३० ईसापूर्व में हुआ था, जिसके बाद इसके प्रदेशों पर यूनानी (मेसीडोन) प्रभुत्व स्थापित हो गया था।
पश्चिम में इस साम्राज्य को मिस्र एवम बेबीलोन पर अधिकार, यूनान के साथ युद्ध तथा यहूदियों के मंदिर निर्माण में सहयोग के लिए याद किया जाता है। कुरोश तथा दारुश को इतिहास में महान की संज्ञा से भी संबोधित किया जाता है। इस वंश को आधुनिक फ़ारसी भाषा बोलने वाले ईरानियों की संस्कृति का आधार कहा जाता है। इस्लाम के पूर्व प्राचीन ईरान के इस साम्राज्य को ईरानी अपने गौरवशाली अतीत की तरह देख

In [142]:
example_id_to_index , features_per_example ,  feature_indices

({'cc4c69225': 0, 'ee569fe45': 1},
 defaultdict(list,
             {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [10, 11, 12, 13, 14, 15]}),
 [10, 11, 12, 13, 14, 15])

##### Exploring

In [134]:
predictions

OrderedDict([('cc4c69225', ' ३३० ईसापूर्व'), ('ee569fe45', ' नेताजी')])

In [132]:
valid_set['PredictionString'] = valid_set['id'].map(predictions)


/tmp/ipykernel_64720/2114411583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_set['PredictionString'] = valid_set['id'].map(predictions)


In [133]:
valid_set

,id,context,question,answer_text,answer_start,language,kfold,answers,PredictionString
0,cc4c69225,हख़ामनी वंश या अजमीढ़ साम्राज्य(अंग्रेज़ी तथा ...,आचमेनिड साम्राज्य का अंत किस साल में हुआ था?,सन ३३० ईसापूर्व,522,hindi,0,"{'answer_start': [522], 'text': ['सन ३३० ईसापू...",३३० ईसापूर्व
1,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,0,"{'answer_start': [2017], 'text': ['नेताजी']}",नेताजी


In [13]:
for data in tqdm(train_dataloader, total=len(train_dataloader)):
    for k, v in data.items():
        if type(v) == list:
#             data[k] = v
            print(k,len(v))
        else:
            print(k,v.shape)
#             data[k] = v.to(config.DEVICE)

100%|██████████| 4/4 [00:04<00:00,  1.16s/it]

input_ids torch.Size([32, 384])
attention_mask torch.Size([32, 384])
offset_mapping torch.Size([32, 384, 2])
start_position torch.Size([32])
end_position torch.Size([32])
context 32
input_ids torch.Size([32, 384])
attention_mask torch.Size([32, 384])
offset_mapping torch.Size([32, 384, 2])
start_position torch.Size([32])
end_position torch.Size([32])
context 32
input_ids torch.Size([32, 384])
attention_mask torch.Size([32, 384])
offset_mapping torch.Size([32, 384, 2])
start_position torch.Size([32])
end_position torch.Size([32])
context 32
input_ids torch.Size([12, 384])
attention_mask torch.Size([12, 384])
offset_mapping torch.Size([12, 384, 2])
start_position torch.Size([12])
end_position torch.Size([12])
context 12


In [21]:
len(data['context'])

12

In [16]:
train_set

,id,context,question,answer_text,answer_start,language,kfold,answers
1084,311617604,मेसोअमेरिका या मेसो-अमेरिका (Spanish: Mesoamér...,मेसोअमेरिका' का यूनानी भाषा में शाब्दिक अर्थ क...,"""मध्य अमेरिका""",704,hindi,0,"{'answer_start': [704], 'text': ['""मध्य अमेरिक..."
1093,871696a7e,प्रोटीन या प्रोभूजिन एक जटिल भूयाति युक्त कार्...,एक ग्राम प्रोटीन के प्रजारण से शरीर को कितनी क...,४.१ कैलीरी,1356,hindi,0,"{'answer_start': [1356], 'text': ['४.१ कैलीरी']}"
1094,2d4a8e922,Main Page\n\nट्वाइलाइट एक रूमानी-फंतासी फ़िल...,2008 की ट्वाइलाइट रूमानी-फंतासी फ़िल्म में निक...,रोज़ाली हेल,4221,hindi,0,"{'answer_start': [4221], 'text': ['रोज़ाली हेल']}"
1096,cc4c69225,हख़ामनी वंश या अजमीढ़ साम्राज्य(अंग्रेज़ी तथा ...,आचमेनिड साम्राज्य का अंत किस साल में हुआ था?,सन ३३० ईसापूर्व,522,hindi,0,"{'answer_start': [522], 'text': ['सन ३३० ईसापू..."


In [15]:
data

{'input_ids': tensor([[     0,   2021,    471,  ..., 125561, 216022,      2],
         [     0,   9163,  30143,  ..., 242374, 163579,      2],
         [     0,   2021,    471,  ..., 123895,   3282,      2],
         ...,
         [     0,   9163,  30143,  ...,    247,   4993,      2],
         [     0,   2021,    471,  ...,   5988,    968,      2],
         [     0,   9163,  30143,  ...,  11526,    125,      2]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 'offset_mapping': tensor([[[    0,     0],
          [    0,     4],
          [    4,     7],
          ...,
          [ 4743,  4750],
          [ 4750,  4756],
          [    0,     0]],
 
         [[    0,     0],
          [    0,     2],
          [    2,     4],
          ...,
          

In [126]:

for i,text in enumerate(train_features):
    print(i,text['context'])
    

0 704
1 704
2 704
3 704
4 704
5 704
6 704
7 704
8 704
9 704
10 704
11 704
12 704
13 704
14 704
15 704
16 704
17 704
18 704
19 704
20 704
21 704
22 704
23 704
24 704
25 704
26 704
27 704
28 704
29 704
30 704
31 704
32 704
33 704
34 704
35 704
36 704
37 704
38 704
39 704
40 704
41 704
42 704
43 704
44 704
45 704
46 704
47 704
48 704
49 704
50 704
51 704
52 704
53 704
54 704
55 704
56 704
57 704
58 704
59 704
60 704
61 704
62 1356
63 1356
64 1356
65 1356
66 4221
67 4221
68 4221
69 4221
70 4221
71 4221
72 4221
73 4221
74 4221
75 4221
76 4221
77 4221
78 4221
79 4221
80 4221
81 4221
82 4221
83 4221
84 4221
85 4221
86 4221
87 4221
88 4221
89 4221
90 4221
91 4221
92 4221
93 4221
94 4221
95 4221
96 4221
97 4221
98 522
99 522
100 522
101 522
102 522
103 522
104 522
105 522
106 522
107 522


In [13]:
next(iter(valid_dataloader))['start_position']

tensor([  0, 360, 119,   0,   0,   0])

In [258]:
c0 = tokenizer.decode(tokenized_example.input_ids[0])
c1 = tokenizer.decode(tokenized_example.input_ids[1])
c2 = tokenizer.decode(tokenized_example.input_ids[2])
c3 = tokenizer.decode(tokenized_example.input_ids[3])
c4 = tokenizer.decode(tokenized_example.input_ids[4])
c5 = tokenizer.decode(tokenized_example.input_ids[5])


In [369]:
offsets = offset_mapping[2]

In [372]:
offsets[119]

(2016, 2021)

In [30]:
for fold in range(2,3): 
    print( fold)

2


In [18]:
cc =True

In [33]:
example_id_to_index = {k: i for i, k in enumerate([12,13,14,15,16])}

In [36]:
example_id_to_index

{12: 0, 13: 1, 14: 2, 15: 3, 16: 4}

In [38]:
import collections

In [39]:
features_per_example = collections.defaultdict(list)
    

In [40]:
features_per_example

defaultdict(list, {})

In [70]:
train_set[train_set.index==7724]

,id,context,question,answer_text,answer_start,language,kfold,answers
7724,6611,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi,-1,"{'answer_start': [328], 'text': ['इलेक्ट्रोस्ट..."


In [69]:
tokenized_example = tokenizer(
        train_set[train_set.index==7724]["question"].values[0],
        train_set[train_set.index==7724]["context"].values[0],
        truncation="only_second",
        max_length=config.MAX_SEQ_LEN,
        stride=config.DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

In [71]:
tokenized_example.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [55]:
for i, feature in enumerate(train_features):
#         features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    print(i,feature.keys())


0 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
1 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
2 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
3 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
4 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
5 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
6 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
7 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
8 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
9 dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])
10 dict_keys(['input_ids', 'at

# check : Try to create a unique key for each tokens - which will help to ensemble the models

In [14]:
new_data = pd.read_csv('input/all_data.csv')
# new_data.answers = new_data.answers.apply(eval)

In [15]:
new_data.tail()

,id,context,question,answer_text,answer_start,language,kfold,answers,external
11291,1762,भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथ...,गहरा चीरा क्या करता है?,"कैरोटिड धमनी, श्वासनली और गले की नसों को काट द...",326,hindi,-1,"{'answer_start': [326], 'text': ['कैरोटिड धमनी...",1
11292,3314,1987 में मार्क नोफ्लेयर ने एकल परियोजनाओं और फ...,नेल्सन मंडेला की 70वीं वर्षगांठ किस वर्ष मनाई गई?,1988,91,hindi,-1,"{'answer_start': [91], 'text': ['1988']}",1
11293,10638,"தொழில்முறை சேவைகளுடன் சேர்ந்து, ஊடக நிறுவனங்கள...",லண்டனின் ஊடக விநியோகத் துறையில் முக்கிய வீரர் ...,பிபிசி,136,tamil,-2,"{'answer_start': [136], 'text': ['பிபிசி']}",1
11294,5523,"रोल्लो के आगमन से पहले, इसकी आबादी पिकार्डी य...",मूल वाइकिंग निवासियों को आगमन के बाद एक सामान्...,रोल्लो,1,hindi,-1,"{'answer_start': [1], 'text': ['रोल्लो']}",1
11295,2065,"मध्य युग में, यह पुराने प्रशिया का त्वांगस्ते ...",2010 में कैलिनिनग्राद की आबादी क्या थी?,"431,902",406,hindi,-1,"{'answer_start': [406], 'text': ['431,902']}",1


In [17]:
new_data = new_data[~new_data.id.isin(ids_to_exclude)]

In [18]:
new_data.shape

(11289, 9)

In [20]:
new_data.kfold.value_counts()

-1    6615
-2    3567
 0     223
 2     223
 1     221
 3     221
 4     219
Name: kfold, dtype: int64

In [27]:
cc =new_data[new_data.kfold.isin([0,1,2,3,4,-1])].reset_index(drop=True)

In [30]:
tamil_data= new_data[new_data.kfold.isin([-2])].reset_index(drop=True)

In [31]:
tamil_data.head()

,id,context,question,answer_text,answer_start,language,kfold,answers,external
0,9413,மைக்ரோனீஸ்ஸியர்கள் 2 வது மில்லினியம் கி.மு.யில...,மார்ஷல் தீவுகள் எப்போது முதலில் தீர்க்கப்பட்டன?,2 வது மில்லினியம் கி.மு,19,tamil,-2,"{'answer_start': [19], 'text': ['2 வது மில்லின...",1
1,8403,பாதுகாப்பு ஊழியர்களின் அமைச்சர்கள் மற்றும் தலை...,"யார் பொதுமக்கள், விஞ்ஞான மற்றும் தொழில்முறை இர...",பாதுகாப்பு ஊழியர்களின் அமைச்சர்கள் மற்றும் தலை...,0,tamil,-2,"{'answer_start': [0], 'text': ['பாதுகாப்பு ஊழி...",1
2,10822,கலிபாவின் வடக்கு-மேற்கு ஆபிரிக்க தளங்களில் இரு...,நர்போனின் நட்பானை நடத்திய இறுதி ஆண்டு என்ன?,759,241,tamil,-2,"{'answer_start': [241], 'text': ['759']}",1
3,8061,"அரசியல் ஊழல் சூழலில், ஒரு லஞ்சம் ஒரு உத்தியோகப...",லஞ்சத்தில் ஈடுபடக்கூடிய குறைந்தபட்ச அளவு என்ன?,இரண்டு,144,tamil,-2,"{'answer_start': [144], 'text': ['இரண்டு']}",1
4,9288,"1997 முதல் 2004 வரை, டெல் நிலையான வளர்ச்சியை அ...",டெல் 1997 முதல் 2004 வரை என்ன கிடைத்தது?,நிலையான வளர்ச்சி,26,tamil,-2,"{'answer_start': [26], 'text': ['நிலையான வளர்ச...",1


In [25]:
from sklearn import model_selection

In [29]:
cc = create_folds(cc,5)

In [35]:
tamil_data['new_kfold'] = -1

In [36]:
all_data = pd.concat([cc,tamil_data])

In [38]:
all_data.tail()

,id,context,question,answer_text,answer_start,language,kfold,answers,external,new_kfold
3562,8030,காத்மண்டுவில் காணப்படும் பெரும்பாலான உணவு வகைக...,என்ன விலங்கு இருந்து வருகிறது?,தண்ணீர் எருமை,248,tamil,-2,"{'answer_start': [248], 'text': ['தண்ணீர் எரும...",1,-1
3563,8520,டர்னர் கிளாசிக் திரைப்படங்கள் உலகம் முழுவதும் ...,இங்கிலாந்தில் இயக்கப்படும் TCM Spinoff சேனலின்...,TCM 2,82,tamil,-2,"{'answer_start': [82], 'text': ['TCM 2']}",1,-1
3564,8648,யுனைடெட் ஸ்டேட் பராமரிக்க கடைசி ஆட்சியாளர் Mst...,இப்பகுதியை ஐக்கியப்படுத்திய கடைசி ஆட்சியாளர் ய...,Mstislav பெரிய,43,tamil,-2,"{'answer_start': [43], 'text': ['Mstislav பெரி...",1,-1
3565,8673,ஆங்கில கால காமிக்ஸ் ஆரம்பகால அமெரிக்க செய்தித்...,நகைச்சுவை வேலைகளைத் தவிர வேறு எந்த வகையான வேலை...,அல்லாத நகைச்சுவை,162,tamil,-2,"{'answer_start': [162], 'text': ['அல்லாத நகைச்...",1,-1
3566,10638,"தொழில்முறை சேவைகளுடன் சேர்ந்து, ஊடக நிறுவனங்கள...",லண்டனின் ஊடக விநியோகத் துறையில் முக்கிய வீரர் ...,பிபிசி,136,tamil,-2,"{'answer_start': [136], 'text': ['பிபிசி']}",1,-1


In [40]:
all_data.to_csv('input/all_data_folds.csv',index=False)

In [23]:
def create_folds(data, num_splits):
    data["new_kfold"] = -1
    kf = model_selection.StratifiedKFold(
        n_splits=num_splits, shuffle=True, random_state=2021)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data['language'])):
        data.loc[v_, 'new_kfold'] = f
    return data

In [16]:
ids_to_exclude = ['1a2160a69','2b41f3744','bc9f0d533','632c16ba0','cd1339bdd','67d7a612e','0745249f0']

In [6]:
new_data.kfold.value_counts()

-1    6615
-2    3567
 0     223
 1     223
 2     223
 3     223
 4     222
Name: kfold, dtype: int64

In [7]:
new_data[new_data.kfold==-2]['language'].value_counts()

tamil    3567
Name: language, dtype: int64

In [5]:
context = new_data.iloc[11291]['context']
question = new_data.iloc[11291]['question']

In [6]:
new_data['len_ans'] = new_data['answer_text'].apply(lambda x: len(x))

In [7]:
del new_data['len_ans']

In [8]:
question ,context 

('गहरा चीरा क्या करता है?',
 'भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग करता है। दबीहा (ذَبِيَحَة) सभी मांस स्रोतों के लिए वध की निर्धारित पद्धति है, मछली और अन्य समुद्री जीवन को छोड़कर, इस्लामी कानून के अनुसार। जानवरों को मारने की इस विधि में तेज, गहरी चीरा बनाने के लिए एक अच्छी तरह से धारदार चाकू का उपयोग किया जाता है जो गले के सामने, कैरोटिड धमनी, श्वासनली और गले की नसों को काट देता है। एक जानवर का सिर जिसे हलाल विधियों का उपयोग करके वध किया जाता है, को चीला के साथ जोड़ दिया जाता है । दिशा के अलावा, अनुमति प्राप्त जानवरों को " ईश्वर के नाम पर " इस्लामिक प्रार्थना बिस्मिल्लाह के उच्चारण पर वध करना चाहिए।')

In [9]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta/deepset/xlm-roberta-large-squad2/")
tokenizer_rem = AutoTokenizer.from_pretrained("Rembert/google/rembert/")

In [11]:
tokenized_example = tokenizer(
    question,
    context,
    truncation="only_second",
    max_length=512,
    stride=192,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)



tokenized_example_rem = tokenizer_rem(
    question,
    context,
    truncation="only_second",
    max_length=512,
    stride=192,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [12]:
tokenized_example.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [13]:
tokenized_example_rem.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [14]:
cc = tokenizer.decode(tokenized_example.input_ids[0])
cc_rem = tokenizer_rem.decode(tokenized_example_rem.input_ids[0])

In [15]:
cc

'<s> गहरा चीरा क्या करता है?</s></s> भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग करता है। दबीहा (ذَبِيَحَة) सभी मांस स्रोतों के लिए वध की निर्धारित पद्धति है, मछली और अन्य समुद्री जीवन को छोड़कर, इस्लामी कानून के अनुसार। जानवरों को मारने की इस विधि में तेज, गहरी चीरा बनाने के लिए एक अच्छी तरह से धारदार चाकू का उपयोग किया जाता है जो गले के सामने, कैरोटिड धमनी, श्वासनली और गले की नसों को काट देता है। एक जानवर का सिर जिसे हलाल विधियों का उपयोग करके वध किया जाता है, को चीला के साथ जोड़ दिया जाता है । दिशा के अलावा, अनुमति प्राप्त जानवरों को " ईश्वर के नाम पर " इस्लामिक प्रार्थना बिस्मिल्लाह के उच्चारण पर वध करना चाहिए।</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [16]:
cc_rem

'[CLS] गहरा चीरा क्या करता है?[SEP] भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग करता है। दबीहा (ذَبِيَحَة) सभी मांस स्रोतों के लिए वध की निर्धारित पद्धति है, मछली और अन्य समुद्री जीवन को छोड़कर, इस्लामी कानून के अनुसार। जानवरों को मारने की इस विधि में तेज, गहरी चीरा बनाने के लिए एक अच्छी तरह से धारदार चाकू का उपयोग किया जाता है जो गले के सामने, कैरोटिड धमनी, श्वासनली और गले की नसों को काट देता है। एक जानवर का सिर जिसे हलाल विधियों का उपयोग करके वध किया जाता है, को चीला के साथ जोड़ दिया जाता है । दिशा के अलावा, अनुमति प्राप्त जानवरों को " ईश्वर के नाम पर " इस्लामिक प्रार्थना बिस्मिल्लाह के उच्चारण पर वध करना चाहिए।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD

In [25]:
tokenizer.decode(tokenized_example.input_ids[0][12:200])

'भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग करता है। दबीहा (ذَبِيَحَة) सभी मांस स्रोतों के लिए वध की निर्धारित पद्धति है, मछली और अन्य समुद्री जीवन को छोड़कर, इस्लामी कानून के अनुसार। जानवरों को मारने की इस विधि में तेज, गहरी चीरा बनाने के लिए एक अच्छी तरह से धारदार चाकू का उपयोग किया जाता है जो गले के सामने, कैरोटिड धमनी, श्वासनली और गले की नसों को काट देता है। एक जानवर का सिर जिसे हलाल विधियों का उपयोग करके वध किया जाता है, को चीला के साथ जोड़ दिया जाता है । दिशा के अलावा, अनुमति प्राप्त जानवरों को " ईश्वर के नाम पर " इस्लामिक प्रार्थना बिस्मिल्लाह के उच्चारण पर वध करना चाहिए'

('output/sentencepiece.bpe.model',)

In [19]:
items= tokenized_example.input_ids[0]

In [20]:
items_rem = tokenized_example_rem.input_ids[0]

In [21]:
tokenizer.convert_ids_to_tokens(items)

['<s>',
 '▁ग',
 'हरा',
 '▁',
 'ची',
 'रा',
 '▁क्या',
 '▁करता',
 '▁है',
 '?',
 '</s>',
 '</s>',
 '▁भोजन',
 '▁एक',
 '▁आपूर्ति',
 'कर्ता',
 '▁से',
 '▁आ',
 'ना',
 '▁चाहिए',
 '▁जो',
 '▁ह',
 'लाल',
 '▁प्र',
 'था',
 'ओं',
 '▁का',
 '▁उपयोग',
 '▁करता',
 '▁है',
 '।',
 '▁द',
 'बी',
 'हा',
 '▁(',
 'ذ',
 'َ',
 'ب',
 'ِ',
 'ي',
 'َ',
 'ح',
 'َ',
 'ة',
 ')',
 '▁सभी',
 '▁',
 'मांस',
 '▁स्रोत',
 'ों',
 '▁के',
 '▁लिए',
 '▁',
 'वध',
 '▁की',
 '▁निर्धारित',
 '▁पद्धत',
 'ि',
 '▁है',
 ',',
 '▁म',
 'छ',
 'ली',
 '▁और',
 '▁अन्य',
 '▁समुद्र',
 'ी',
 '▁जीवन',
 '▁को',
 '▁छोड़',
 'कर',
 ',',
 '▁इस्लाम',
 'ी',
 '▁कानून',
 '▁के',
 '▁अनुसार',
 '।',
 '▁जान',
 'वर',
 'ों',
 '▁को',
 '▁मार',
 'ने',
 '▁की',
 '▁इस',
 '▁विधि',
 '▁में',
 '▁तेज',
 ',',
 '▁ग',
 'हरी',
 '▁',
 'ची',
 'रा',
 '▁बनाने',
 '▁के',
 '▁लिए',
 '▁एक',
 '▁अच्छी',
 '▁तरह',
 '▁से',
 '▁',
 'धार',
 'दार',
 '▁चा',
 'कू',
 '▁का',
 '▁उपयोग',
 '▁किया',
 '▁जाता',
 '▁है',
 '▁जो',
 '▁ग',
 'ले',
 '▁के',
 '▁सामने',
 ',',
 '▁कै',
 'रो',
 'टि',
 'ड',
 '▁',
 'धम',
 'नी',
 

In [77]:
tokenizer_rem.convert_ids_to_tokens(items_rem)

['[CLS]',
 '▁ग',
 'हरा',
 '▁ची',
 'रा',
 '▁क्या',
 '▁करता',
 '▁है',
 '?',
 '[SEP]',
 '▁भोजन',
 '▁एक',
 '▁आपूर्ति',
 'कर्ता',
 '▁से',
 '▁आ',
 'ना',
 '▁चाहिए',
 '▁जो',
 '▁ह',
 'लाल',
 '▁प्र',
 'था',
 'ओं',
 '▁का',
 '▁उपयोग',
 '▁करता',
 '▁है',
 '।',
 '▁द',
 'बी',
 'हा',
 '▁(',
 'ذ',
 'َ',
 'ب',
 'ِ',
 'ي',
 'َ',
 'ح',
 'َ',
 'ة',
 ')',
 '▁सभी',
 '▁मा',
 'ंस',
 '▁स्रोत',
 'ों',
 '▁के',
 '▁लिए',
 '▁',
 'वध',
 '▁की',
 '▁निर्धारित',
 '▁पद्धति',
 '▁है',
 ',',
 '▁म',
 'छ',
 'ली',
 '▁और',
 '▁अन्य',
 '▁समुद्र',
 'ी',
 '▁जीवन',
 '▁को',
 '▁छोड़',
 'कर',
 ',',
 '▁इस्लाम',
 'ी',
 '▁कानून',
 '▁के',
 '▁अनुसार',
 '।',
 '▁जान',
 'वर',
 'ों',
 '▁को',
 '▁मार',
 'ने',
 '▁की',
 '▁इस',
 '▁विधि',
 '▁में',
 '▁तेज',
 ',',
 '▁गह',
 'री',
 '▁ची',
 'रा',
 '▁बनाने',
 '▁के',
 '▁लिए',
 '▁एक',
 '▁अच्छी',
 '▁तरह',
 '▁से',
 '▁धा',
 'रदार',
 '▁चा',
 'कू',
 '▁का',
 '▁उपयोग',
 '▁किया',
 '▁जाता',
 '▁है',
 '▁जो',
 '▁',
 'गले',
 '▁के',
 '▁सामने',
 ',',
 '▁कै',
 'रो',
 'ट',
 'िड',
 '▁ध',
 'मनी',
 ',',
 '▁',
 'श्वास',
 'न',
 'ली

In [23]:
splitted_context = context.split()
splitted_question = question.split()

In [30]:
context

'भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग करता है। दबीहा (ذَبِيَحَة) सभी मांस स्रोतों के लिए वध की निर्धारित पद्धति है, मछली और अन्य समुद्री जीवन को छोड़कर, इस्लामी कानून के अनुसार। जानवरों को मारने की इस विधि में तेज, गहरी चीरा बनाने के लिए एक अच्छी तरह से धारदार चाकू का उपयोग किया जाता है जो गले के सामने, कैरोटिड धमनी, श्वासनली और गले की नसों को काट देता है। एक जानवर का सिर जिसे हलाल विधियों का उपयोग करके वध किया जाता है, को चीला के साथ जोड़ दिया जाता है । दिशा के अलावा, अनुमति प्राप्त जानवरों को " ईश्वर के नाम पर " इस्लामिक प्रार्थना बिस्मिल्लाह के उच्चारण पर वध करना चाहिए।'

In [58]:
len(items), len(items_rem)

(512, 512)

In [64]:
items[7:20],items_rem[7:20]

([18224, 460, 32, 2, 2, 131409, 967, 150469, 62949, 646, 2220, 1748, 23508],
 [1002,
  628,
  313,
  123794,
  1430,
  237493,
  88306,
  1251,
  2633,
  2927,
  32518,
  4279,
  5528])

In [65]:
tokenizer.decode(items[13:25])

'एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथा'

In [66]:
tokenizer_rem.decode(items_rem[13:25])

'कर्ता से आना चाहिए जो हलाल प्रथाओं का'

In [35]:
items_deduped= list(dict.fromkeys(items))
items_deduped_rem= list(dict.fromkeys(items_rem))

In [34]:
tokenizer.decode(items_deduped)

'<s> गहरा चीरा क्या करता है?</s> भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग। दबीहा (ذَبِيحة) सभीमांस स्रोतों के लिएवध की निर्धारित पद्धति, मछली और अन्य समुद्री जीवन को छोड़कर इस्लाम कानून अनुसार जानवर मारने इस विधि में तेजहरी बनाने अच्छी तरहधारदार चाकू किया जाताले सामने कैरोटिडधमनीश्वासन नस काट देता सिर जिसेयों करके चीला साथ जोड़ दिया । दिशा अलावा अनुमति प्राप्त " ईश्वर नाम परिक प्रार्थना बिस्मिल्लाह उच्चारण करना<pad>'

In [36]:
tokenizer_rem.decode(items_deduped_rem)

'[CLS] गहरा चीरा क्या करता है?[SEP] भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथाओं का उपयोग। दबीहा (ذَبِيحة) सभी मांस स्रोतों के लिए वध की निर्धारित पद्धति, मछली और अन्य समुद्री जीवन को छोड़कर इस्लाम कानून अनुसार जानवर मारने इस विधि में तेज गहरी बनाने अच्छी तरह धारदार चाकू किया जातागले सामने कैरोटिड धमनीश्वासन नस काट देता सिर जिसे विधियों करकेला साथ जोड़ दिया । दिशा अलावा अनुमति प्राप्त " ईश्वर नाम परिकार्थ बिस्मिल्लाह उच्चारण करना[PAD]'

In [37]:
items_deduped_rem

[312,
 7280,
 57428,
 81155,
 6593,
 19768,
 15689,
 1002,
 628,
 313,
 123794,
 1430,
 237493,
 88306,
 1251,
 2633,
 2927,
 32518,
 4279,
 5528,
 78472,
 5602,
 32728,
 9121,
 1257,
 25405,
 661,
 6760,
 13285,
 17236,
 579,
 4397,
 944,
 2084,
 1332,
 974,
 4076,
 1089,
 582,
 20377,
 5245,
 14253,
 117821,
 1703,
 847,
 2655,
 573,
 198625,
 1130,
 187028,
 163977,
 571,
 5491,
 15038,
 3801,
 1325,
 11371,
 89926,
 1199,
 19826,
 1274,
 67449,
 4777,
 135699,
 96505,
 16726,
 24900,
 9205,
 47591,
 2190,
 3229,
 82023,
 969,
 72328,
 120068,
 4996,
 53787,
 100719,
 18174,
 53407,
 137752,
 22245,
 64328,
 3239,
 6926,
 227335,
 66396,
 15607,
 14122,
 2799,
 60247,
 18375,
 198232,
 226091,
 1702,
 53624,
 153968,
 65690,
 98048,
 54583,
 174321,
 6709,
 60738,
 2897,
 4661,
 103783,
 8525,
 1558,
 100943,
 50804,
 161646,
 17467,
 605,
 161023,
 7857,
 1729,
 3890,
 47604,
 10260,
 85850,
 4764,
 214598,
 213070,
 19240,
 0]

In [36]:
import config
import dataset
import preprocessing
from transformers import AutoTokenizer
from model import XLM_RoBertamodel
# import engine


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
device = config.DEVICE #torch.device("cuda:3")  #
model_config = AutoConfig.from_pretrained("Rembert/google/rembert/")
model = XLM_RoBertamodel("Rembert/google/rembert/", config=model_config)
model = nn.DataParallel(model)

# model.cuda();
model.to(device)
model.load_state_dict(torch.load('output/rembert_v3/checkpoint-fold-1/pytorch_model.bin',map_location=torch.device('cuda')));


model.eval()

DataParallel(
  (module): XLM_RoBertamodel(
    (xlm_roberta): RemBertModel(
      (embeddings): RemBertEmbeddings(
        (word_embeddings): Embedding(250300, 256, padding_idx=0)
        (position_embeddings): Embedding(512, 256)
        (token_type_embeddings): Embedding(2, 256)
        (LayerNorm): LayerNorm((256,), eps=1e-07, elementwise_affine=True)
        (dropout): Dropout(p=0, inplace=False)
      )
      (encoder): RemBertEncoder(
        (embedding_hidden_mapping_in): Linear(in_features=256, out_features=1152, bias=True)
        (layer): ModuleList(
          (0): RemBertLayer(
            (attention): RemBertAttention(
              (self): RemBertSelfAttention(
                (query): Linear(in_features=1152, out_features=1152, bias=True)
                (key): Linear(in_features=1152, out_features=1152, bias=True)
                (value): Linear(in_features=1152, out_features=1152, bias=True)
                (dropout): Dropout(p=0, inplace=False)
              )
       

In [8]:
new_data.iloc[11291]

id                                                           1762
context         भोजन एक आपूर्तिकर्ता से आना चाहिए जो हलाल प्रथ...
question                                  गहरा चीरा क्या करता है?
answer_text     कैरोटिड धमनी, श्वासनली और गले की नसों को काट द...
answer_start                                                  326
language                                                    hindi
kfold                                                          -1
answers         {'answer_start': [326], 'text': ['कैरोटिड धमनी...
external                                                        1
Name: 11291, dtype: object

In [64]:
valid_features = []

for i, row in new_data.head(3).iterrows():
    
        valid_features += preprocessing.get_features(row, tokenizer)
    
    
        
valid_dataloader = dataset.Datasetloader(features=valid_features, test=True,inference=True).fetch(
        batch_size=32, num_workers=1, shuffle=False, drop_last=False)
 

In [65]:
next(iter(valid_dataloader))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'input_ids': tensor([[     0, 153493,  44399,  ...,      1,      1,      1],
         [     0,  11674,      5,  ...,  22727, 190324,      2],
         [     0,  11674,      5,  ...,  11449,   5270,      2],
         ...,
         [     0,  11674,      5,  ...,   8838,  12152,      2],
         [     0,  11674,      5,  ...,      1,      1,      1],
         [     0,  16866,   3801,  ...,      1,      1,      1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'offset_mapping': tensor([[[   0,    0],
          [   0,    4],
          [   4,    5],
          ...,
          [   0,    0],
          [   0,    0],
          [   0,    0]],
 
         [[   0,    0],
          [   0,    2],
          [   2,    3],
          ...,
          [1809, 1812],
          [1812, 1819],
          [   0,    0]

In [12]:
valid_features[0].keys()

dict_keys(['context', 'answer_text', 'example_id', 'sequence_ids', 'question', 'input_ids', 'attention_mask', 'offset_mapping', 'start_position', 'end_position'])

In [73]:
start_logits,end_logits = [],[]
outputs_start_ , outputs_end_ = [], []
jaccard_score = 0.0
ans_pred = []
with torch.no_grad():
        
    for data in tqdm(valid_dataloader, total=len(valid_dataloader)):
        for k, v in data.items():
            if isinstance(v,list):
                data[k] = v
            else:
                data[k] = v.to(device)

        outputs_start, outputs_end, loss = model(data['input_ids'],data['attention_mask'])
        start_logits.append(outputs_start.cpu().numpy().tolist())
        end_logits.append(outputs_end.cpu().numpy().tolist())
        
        outputs_start_.append(torch.softmax(outputs_start, dim=-1).cpu().detach().numpy())
        outputs_end_.append(torch.softmax(outputs_end, dim=-1).cpu().detach().numpy())



  0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


In [74]:
import copy

In [75]:
valid_set = new_data.head(3)

In [76]:
valid_features_ =  copy.deepcopy(valid_features)
jaccard , valid_ans  = preprocessing.postprocess_qa_predictions(examples=valid_set, features=valid_features_,raw_predictions=(start_logits,end_logits) ,tokenizer = tokenizer, n_best_size = 20, max_answer_length = 30)

Post-processing 3 example predictions split into 8 features.


TypeError: '>=' not supported between instances of 'list' and 'int'

In [67]:
outputs_start_[0][0]

array([0.00129414, 0.00135703, 0.00037225, 0.00151008, 0.00158924,
       0.00055133, 0.00082892, 0.00084295, 0.00114497, 0.00175418,
       0.00037429, 0.0012239 , 0.00069652, 0.00725499, 0.00126057,
       0.00084584, 0.00036433, 0.00368287, 0.00120846, 0.00069207,
       0.00420258, 0.00237477, 0.00090878, 0.00064236, 0.01114245,
       0.00080437, 0.00042307, 0.00056619, 0.000726  , 0.00043938,
       0.00174004, 0.00203779, 0.00095051, 0.00048163, 0.00053504,
       0.00055134, 0.00084711, 0.00033158, 0.00108958, 0.00116485,
       0.00062604, 0.00133015, 0.00145764, 0.00034553, 0.00104356,
       0.00219076, 0.00052772, 0.00064367, 0.00083016, 0.001276  ,
       0.00088484, 0.0012126 , 0.00062751, 0.00213102, 0.00057447,
       0.00106876, 0.00216987, 0.00139898, 0.00045249, 0.00055904,
       0.00135353, 0.0005475 , 0.00265227, 0.00085801, 0.00118758,
       0.00040263, 0.00122973, 0.00171015, 0.00153997, 0.00277429,
       0.0003951 , 0.00040796, 0.00250415, 0.00170297, 0.00118

In [58]:
len(start_logits[0])

10

In [62]:
## `load both the models and add the logits and see from there
new_data.head()


,id,context,question,answer_text,answer_start,language,kfold,answers,external
0,9413,மைக்ரோனீஸ்ஸியர்கள் 2 வது மில்லினியம் கி.மு.யில...,மார்ஷல் தீவுகள் எப்போது முதலில் தீர்க்கப்பட்டன?,2 வது மில்லினியம் கி.மு,19,tamil,-2,"{'answer_start': [19], 'text': ['2 வது மில்லின...",1
1,f34e9ce70,சக்கரவர்த்தி இராஜகோபாலாச்சாரி (Chakravarti Raj...,சி.ஆர்.ராவ் எப்போது பிறந்தார்?,10 திசம்பர் 1878,60,tamil,4,"{'answer_start': [60], 'text': ['10 திசம்பர் 1...",0
2,2782,अमेथिस्ट का प्राचीन मिस्रवासियों द्वारा रत्न क...,मोती किस चीज से बनते हैं?,अमेथिस्ट,454,hindi,-1,"{'answer_start': [454], 'text': ['अमेथिस्ट']}",1
3,885,परिवार नियोजन की नैतिकता को लेकर धर्मों के विच...,किस धर्म का मानना है कि गर्भाधान को रोकना स्वी...,बौद्ध,567,hindi,-1,"{'answer_start': [567], 'text': ['बौद्ध']}",1
4,8403,பாதுகாப்பு ஊழியர்களின் அமைச்சர்கள் மற்றும் தலை...,"யார் பொதுமக்கள், விஞ்ஞான மற்றும் தொழில்முறை இர...",பாதுகாப்பு ஊழியர்களின் அமைச்சர்கள் மற்றும் தலை...,0,tamil,-2,"{'answer_start': [0], 'text': ['பாதுகாப்பு ஊழி...",1
